In [1]:
import pandas as pd
import re 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fnmatch
from yelpapi import YelpAPI
import json
from decimal import *

%matplotlib inline

In [2]:
pov_data = pd.read_excel('../Data/poverty_rates.xlsx', sheet_name = 'Percents', header = 2)
pov_data

#bringing in data from census

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,0.026037,0.012483,0.045000,0.004918,0.034508,0.011601,0.034091,0.008723,...,0.036052,0.009398,0.024596,0.005037,0.036945,0.007138,0.012765,0.003430,0.028517,0.009377
1,.50 to .74,poverty,0.024342,0.012483,0.023456,0.003069,0.022094,0.010641,0.016845,0.005842,...,0.019741,0.007608,0.021070,0.004877,0.014934,0.004207,0.010619,0.003978,0.011689,0.004958
2,.75 to .99,poverty,0.022202,0.008382,0.037121,0.003978,0.026306,0.010788,0.023047,0.007602,...,0.034112,0.011686,0.020271,0.004767,0.024564,0.005882,0.008348,0.003414,0.015594,0.005010
3,1.00 to 1.24,low-income,0.030584,0.012305,0.031564,0.003722,0.026528,0.010641,0.041693,0.013684,...,0.021880,0.006415,0.024117,0.004423,0.023726,0.005643,0.021285,0.006578,0.020476,0.006423
4,1.25 to 1.49,low-income,0.033081,0.010700,0.039468,0.004527,0.027562,0.009163,0.030210,0.010083,...,0.039582,0.011885,0.034043,0.007765,0.029409,0.006859,0.010087,0.002945,0.023045,0.006860
5,1.50 to 1.74,low-income,0.038698,0.012037,0.038541,0.004502,0.034360,0.010345,0.048496,0.013404,...,0.032074,0.009597,0.026598,0.005283,0.027834,0.005523,0.012201,0.003759,0.025152,0.007039
6,1.75 to 1.84,low-income,0.019082,0.012127,0.016473,0.002711,0.020838,0.011158,0.018326,0.008083,...,0.013526,0.007260,0.014006,0.004374,0.010906,0.003370,0.008896,0.002631,0.009223,0.003468
7,1.85 to 1.99,low-income,0.029157,0.010878,0.020719,0.003376,0.031848,0.011158,0.025768,0.008283,...,0.021929,0.006663,0.024473,0.004755,0.020656,0.005762,0.012358,0.006061,0.013976,0.003879
8,2.00 to 2.99,NaN,0.169327,0.021400,0.146650,0.007731,0.202172,0.029262,0.197423,0.023648,...,0.179712,0.022576,0.178449,0.014190,0.147064,0.012102,0.086676,0.009163,0.156998,0.018035
9,3.00 to 3.99,NaN,0.192867,0.024253,0.134462,0.007917,0.196483,0.031848,0.165773,0.021407,...,0.167330,0.022327,0.161986,0.009325,0.172346,0.014156,0.099018,0.010431,0.156073,0.015004


In [3]:
pov_data.info()   #making sure numbers are the correct type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rate                   13 non-null     object 
 1   Income level           8 non-null      object 
 2   Cheatham County, TN    13 non-null     float64
 3   Error                  13 non-null     float64
 4   Davidson County, TN    13 non-null     float64
 5   Error.1                13 non-null     float64
 6   Dickson County, TN     13 non-null     float64
 7   Error.2                13 non-null     float64
 8   Maury County, TN       13 non-null     float64
 9   Error.3                13 non-null     float64
 10  Montgomery County, TN  13 non-null     float64
 11  Error.4                13 non-null     float64
 12  Robertson County, TN   13 non-null     float64
 13  Error.5                13 non-null     float64
 14  Rutherford County, TN  13 non-null     float64
 15  Error.6 

In [4]:
pov_data[['Cheatham County, TN', 'Error', 'Davidson County, TN', 'Error.1', 'Dickson County, TN', 'Error.2', 'Maury County, TN', 'Error.3', 'Montgomery County, TN', 'Error.4', 'Robertson County, TN', 'Error.5', 'Rutherford County, TN', 'Error.6', 'Sumner County, TN', 'Error.7', 'Williamson County, TN', 'Error.8', 'Wilson County, TN', 'Error.9']] = pov_data[['Cheatham County, TN', 'Error', 'Davidson County, TN', 'Error.1', 'Dickson County, TN', 'Error.2', 'Maury County, TN', 'Error.3', 'Montgomery County, TN', 'Error.4', 'Robertson County, TN', 'Error.5', 'Rutherford County, TN', 'Error.6', 'Sumner County, TN', 'Error.7', 'Williamson County, TN', 'Error.8', 'Wilson County, TN', 'Error.9']].apply(lambda x: x*100)
pov_data

#updating column titles

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,2.603656,1.248328,4.499965,0.491754,3.450824,1.160127,3.409091,0.872279,...,3.605172,0.939831,2.459580,0.503710,3.694547,0.713787,1.276489,0.343008,2.851711,0.937725
1,.50 to .74,poverty,2.434240,1.248328,2.345583,0.306947,2.209414,1.064066,1.684539,0.584187,...,1.974142,0.760816,2.106983,0.487739,1.493371,0.420696,1.061914,0.397826,1.168945,0.495838
2,.75 to .99,poverty,2.220241,0.838163,3.712135,0.397752,2.630607,1.078844,2.304738,0.760243,...,3.411238,1.168573,2.027127,0.476682,2.456385,0.588177,0.834808,0.341441,1.559449,0.500976
3,1.00 to 1.24,low-income,3.058404,1.230495,3.156434,0.372173,2.652775,1.064066,4.169334,1.368438,...,2.187966,0.641472,2.411666,0.442282,2.372645,0.564251,2.128526,0.657823,2.047580,0.642277
4,1.25 to 1.49,low-income,3.308070,1.069996,3.946822,0.452746,2.756226,0.916279,3.020967,1.008323,...,3.958230,1.188463,3.404344,0.776451,2.940883,0.685874,1.008661,0.294454,2.304491,0.685952
5,1.50 to 1.74,low-income,3.869817,1.203745,3.854098,0.450188,3.436045,1.034508,4.849552,1.340429,...,3.207360,0.959722,2.659836,0.528281,2.783372,0.552288,1.220104,0.375899,2.515158,0.703936
6,1.75 to 1.84,low-income,1.908159,1.212662,1.647280,0.271136,2.083795,1.115791,1.832586,0.808259,...,1.352561,0.726007,1.400560,0.437368,1.090619,0.336955,0.889627,0.263129,0.922310,0.346830
7,1.85 to 1.99,low-income,2.915738,1.087829,2.071889,0.337641,3.184808,1.115791,2.576825,0.828265,...,2.192939,0.666335,2.447295,0.475453,2.065597,0.576214,1.235767,0.606137,1.397595,0.387935
8,2.00 to 2.99,NaN,16.932679,2.139991,14.665013,0.773122,20.217247,2.926180,19.742318,2.364757,...,17.971159,2.257583,17.844857,1.418989,14.706410,1.210248,8.667596,0.916253,15.699825,1.803515
9,3.00 to 3.99,NaN,19.286670,2.425323,13.446179,0.791666,19.648267,3.184808,16.577305,2.140685,...,16.732969,2.232720,16.198585,0.932478,17.234573,1.415612,9.901796,1.043119,15.607337,1.500360


In [5]:
pov_data.round(2)  #rounding numbers to 2 decimal places

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,2.60,1.25,4.50,0.49,3.45,1.16,3.41,0.87,...,3.61,0.94,2.46,0.50,3.69,0.71,1.28,0.34,2.85,0.94
1,.50 to .74,poverty,2.43,1.25,2.35,0.31,2.21,1.06,1.68,0.58,...,1.97,0.76,2.11,0.49,1.49,0.42,1.06,0.40,1.17,0.50
2,.75 to .99,poverty,2.22,0.84,3.71,0.40,2.63,1.08,2.30,0.76,...,3.41,1.17,2.03,0.48,2.46,0.59,0.83,0.34,1.56,0.50
3,1.00 to 1.24,low-income,3.06,1.23,3.16,0.37,2.65,1.06,4.17,1.37,...,2.19,0.64,2.41,0.44,2.37,0.56,2.13,0.66,2.05,0.64
4,1.25 to 1.49,low-income,3.31,1.07,3.95,0.45,2.76,0.92,3.02,1.01,...,3.96,1.19,3.40,0.78,2.94,0.69,1.01,0.29,2.30,0.69
5,1.50 to 1.74,low-income,3.87,1.20,3.85,0.45,3.44,1.03,4.85,1.34,...,3.21,0.96,2.66,0.53,2.78,0.55,1.22,0.38,2.52,0.70
6,1.75 to 1.84,low-income,1.91,1.21,1.65,0.27,2.08,1.12,1.83,0.81,...,1.35,0.73,1.40,0.44,1.09,0.34,0.89,0.26,0.92,0.35
7,1.85 to 1.99,low-income,2.92,1.09,2.07,0.34,3.18,1.12,2.58,0.83,...,2.19,0.67,2.45,0.48,2.07,0.58,1.24,0.61,1.40,0.39
8,2.00 to 2.99,NaN,16.93,2.14,14.67,0.77,20.22,2.93,19.74,2.36,...,17.97,2.26,17.84,1.42,14.71,1.21,8.67,0.92,15.70,1.80
9,3.00 to 3.99,NaN,19.29,2.43,13.45,0.79,19.65,3.18,16.58,2.14,...,16.73,2.23,16.20,0.93,17.23,1.42,9.90,1.04,15.61,1.50


In [6]:
with open('keys.json') as fi:
    credentials = json.load(fi)
    
#calling in yelp api and secret id

In [7]:
api_key = credentials['yelp_key']
client_id = credentials['yelp_id']
yelp_api = YelpAPI(api_key)

In [8]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
parks = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'park', location = location, search_limit = 20, offset = x)
        parks.append(response)
        print(len(parks))
        
#creating loop. made list of locations to run through. group is how many time to run through for the offset (only gives 20
# without offset changing). created list of parks to store found results in. 
#loop goes through to find parks in all the locations listed and is going through yelp until reaching the 500 limit
#then appending it to add it each loop to the parks list.
#printing list to check that it is changing after each loop

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [9]:
parks[1]     #checking to see what the parks list looks like. It is a list of dicts of 20 parks

{'businesses': [{'id': 'G3FSsyHWzJ-s4ndaC-C95Q',
   'alias': 'henslee-park-dickson',
   'name': 'Henslee Park',
   'image_url': '',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/henslee-park-dickson?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 1,
   'categories': [{'alias': 'parks', 'title': 'Parks'}],
   'rating': 5.0,
   'coordinates': {'latitude': 36.087676, 'longitude': -87.397686},
   'transactions': [],
   'location': {'address1': '800 Hwy 70 W',
    'address2': '',
    'address3': None,
    'city': 'Dickson',
    'zip_code': '37055',
    'country': 'US',
    'state': 'TN',
    'display_address': ['800 Hwy 70 W', 'Dickson, TN 37055']},
   'phone': '',
   'display_phone': '',
   'distance': 32357.461332730283},
  {'id': 'yu9e-_S7U97kleJQw0eOzQ',
   'alias': 'richland-creek-greenway-nashville',
   'name': 'Richland Creek Greenway',
   'image_url': 'https://s3-med

In [10]:
parks_df = pd.DataFrame()

for park in range(len(parks)):
    df = pd.DataFrame(parks[park]['businesses'])
    parks_df = parks_df.append(df)
    print(parks_df.shape)
    
#creating a df for response to go into
#looking for parks in the range of the length of parks
#putting each dict within park into a df
#then appending parks_df so each dict that was turned into a df goes into the new parks_df each loop
#printing shape to see that it is grabbing more dicts from parks

(20, 15)
(40, 15)
(60, 15)
(80, 15)
(100, 15)
(120, 15)
(140, 16)
(160, 16)
(180, 16)
(200, 16)
(220, 16)
(240, 16)
(260, 16)
(280, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(308, 16)
(328, 16)
(348, 16)
(368, 16)
(388, 16)
(408, 16)
(428, 16)
(448, 16)
(468, 16)
(488, 16)
(508, 16)
(528, 16)
(548, 16)
(568, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(592, 16)
(612, 16)
(632, 16)
(652, 16)
(672, 16)
(692, 16)
(712, 16)
(732, 16)
(752, 16)
(772, 16)
(792, 16)
(812, 16)
(832, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(840, 16)
(860, 16)
(880, 16)
(900, 16)
(920, 16)
(940, 16)
(960, 16)
(980, 16)
(1000, 16)
(1020, 16)
(1040, 16)
(1060, 16)
(1080, 16)
(1100, 16)
(1101, 16)
(1101, 16)
(1101, 16)
(1101, 16)
(1101, 16)
(1101, 16)
(1101, 16)
(1101, 16)
(1101, 16)
(1101, 16)
(1101, 1

In [11]:
parks_df   #checking to ensure it is a df and running correctly/how many rows

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,docjbpsuMVzYyF-8TVEEBQ,rockland-recreation-area-hendersonville,Rockland Recreation Area,https://s3-media1.fl.yelpcdn.com/bphoto/7ePq3z...,False,https://www.yelp.com/biz/rockland-recreation-a...,4,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.3009417057037, 'longitude': -8...",[],"{'address1': '5 Power Plant Rd', 'address2': '...",+16158224846,(615) 822-4846,40767.341539,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,9zS8gPBAjvg_ckSXMGH-sQ,commerce-center-park-nashville,Commerce Center Park,https://s3-media1.fl.yelpcdn.com/bphoto/YgXAnH...,False,https://www.yelp.com/biz/commerce-center-park-...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.16222, 'longitude': -86.7761002}",[],"{'address1': '120 3rd Ave N', 'address2': '', ...",,,42219.480751,NaN
8,pDqFwkWgZ7L7vE5fGEEfRg,church-st-park-nashville,Church St. Park,,False,https://www.yelp.com/biz/church-st-park-nashvi...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.1626930236816, 'longitude': -8...",[],"{'address1': '600 Church St', 'address2': '', ...",,,42749.900544,NaN
9,aNEiOU-rgYjqM6J2AT6AoA,tango-in-the-park-nashville,Tango in the Park,,False,https://www.yelp.com/biz/tango-in-the-park-nas...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.14943, 'longitude': -86.81293}",[],"{'address1': '2500 West End Avenue', 'address2...",+16153673878,(615) 367-3878,45522.670932,NaN
10,SabmUBdLlLlwDTd1NB97BA,owen-peabody-park-nashville,Owen Peabody Park,https://s3-media3.fl.yelpcdn.com/bphoto/mL9bD7...,False,https://www.yelp.com/biz/owen-peabody-park-nas...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.15216, 'longitude': -86.79135}",[],"{'address1': 'Buddy Killen Cir', 'address2': '...",,,43633.736720,NaN


In [12]:
parks_df = parks_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,docjbpsuMVzYyF-8TVEEBQ,rockland-recreation-area-hendersonville,Rockland Recreation Area,https://s3-media1.fl.yelpcdn.com/bphoto/7ePq3z...,False,https://www.yelp.com/biz/rockland-recreation-a...,4,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.3009417057037, 'longitude': -8...",[],"{'address1': '5 Power Plant Rd', 'address2': '...",+16158224846,(615) 822-4846,40767.341539,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,9zS8gPBAjvg_ckSXMGH-sQ,commerce-center-park-nashville,Commerce Center Park,https://s3-media1.fl.yelpcdn.com/bphoto/YgXAnH...,False,https://www.yelp.com/biz/commerce-center-park-...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.16222, 'longitude': -86.7761002}",[],"{'address1': '120 3rd Ave N', 'address2': '', ...",,,42219.480751,NaN
2764,pDqFwkWgZ7L7vE5fGEEfRg,church-st-park-nashville,Church St. Park,,False,https://www.yelp.com/biz/church-st-park-nashvi...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.1626930236816, 'longitude': -8...",[],"{'address1': '600 Church St', 'address2': '', ...",,,42749.900544,NaN
2765,aNEiOU-rgYjqM6J2AT6AoA,tango-in-the-park-nashville,Tango in the Park,,False,https://www.yelp.com/biz/tango-in-the-park-nas...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.14943, 'longitude': -86.81293}",[],"{'address1': '2500 West End Avenue', 'address2...",+16153673878,(615) 367-3878,45522.670932,NaN
2766,SabmUBdLlLlwDTd1NB97BA,owen-peabody-park-nashville,Owen Peabody Park,https://s3-media3.fl.yelpcdn.com/bphoto/mL9bD7...,False,https://www.yelp.com/biz/owen-peabody-park-nas...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.15216, 'longitude': -86.79135}",[],"{'address1': 'Buddy Killen Cir', 'address2': '...",,,43633.736720,NaN


In [13]:
parks_df[parks_df.duplicated(subset = 'name', keep = 'first')]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
93,PGY64SyinTS5jm-YgMb-9g,liberty-park-franklin,Liberty Park,https://s3-media2.fl.yelpcdn.com/bphoto/U1El6l...,False,https://www.yelp.com/biz/liberty-park-franklin...,5,"[{'alias': 'discgolf', 'title': 'Disc Golf'}]",3.5,"{'latitude': 35.9309170888114, 'longitude': -8...",[],"{'address1': '1669 McEwen Dr', 'address2': '',...",,,44090.968033,NaN
272,9kwQHZ28oi2wnVjpoV-eVw,williamson-county-parks-and-rec-dept-brentwood,Williamson County Parks & Rec Dept,https://s3-media3.fl.yelpcdn.com/bphoto/dvt489...,False,https://www.yelp.com/biz/williamson-county-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",3.5,"{'latitude': 36.0019912719727, 'longitude': -8...",[],"{'address1': '920 Heritage Way', 'address2': '...",+16153703471,(615) 370-3471,38874.553363,NaN
288,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,4675.104705,NaN
289,yu9e-_S7U97kleJQw0eOzQ,richland-creek-greenway-nashville,Richland Creek Greenway,https://s3-media4.fl.yelpcdn.com/bphoto/fiqTzB...,False,https://www.yelp.com/biz/richland-creek-greenw...,30,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.5,"{'latitude': 36.140481933298005, 'longitude': ...",[],"{'address1': '4601 Murphy Rd', 'address2': '',...",,,7350.416823,NaN
290,Wjv13Wi_9RPRJPg-uMkcRQ,bicentennial-capitol-mall-state-park-nashville-2,Bicentennial Capitol Mall State Park,https://s3-media4.fl.yelpcdn.com/bphoto/I5sZVJ...,False,https://www.yelp.com/biz/bicentennial-capitol-...,86,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.1680549, 'longitude': -86.7863...",[],"{'address1': '600 James Robertson Pkwy', 'addr...",+16157415280,(615) 741-5280,2003.591226,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,9zS8gPBAjvg_ckSXMGH-sQ,commerce-center-park-nashville,Commerce Center Park,https://s3-media1.fl.yelpcdn.com/bphoto/YgXAnH...,False,https://www.yelp.com/biz/commerce-center-park-...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.16222, 'longitude': -86.7761002}",[],"{'address1': '120 3rd Ave N', 'address2': '', ...",,,42219.480751,NaN
2764,pDqFwkWgZ7L7vE5fGEEfRg,church-st-park-nashville,Church St. Park,,False,https://www.yelp.com/biz/church-st-park-nashvi...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.1626930236816, 'longitude': -8...",[],"{'address1': '600 Church St', 'address2': '', ...",,,42749.900544,NaN
2765,aNEiOU-rgYjqM6J2AT6AoA,tango-in-the-park-nashville,Tango in the Park,,False,https://www.yelp.com/biz/tango-in-the-park-nas...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.14943, 'longitude': -86.81293}",[],"{'address1': '2500 West End Avenue', 'address2...",+16153673878,(615) 367-3878,45522.670932,NaN
2766,SabmUBdLlLlwDTd1NB97BA,owen-peabody-park-nashville,Owen Peabody Park,https://s3-media3.fl.yelpcdn.com/bphoto/mL9bD7...,False,https://www.yelp.com/biz/owen-peabody-park-nas...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.15216, 'longitude': -86.79135}",[],"{'address1': 'Buddy Killen Cir', 'address2': '...",,,43633.736720,NaN


In [14]:
parks_df[parks_df['name']== 'Shelby Park']   #checking random park to see if duplicates should be deleted
                                             #did this with a few parks. all have multiple repeats

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
39,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,33592.003207,NaN
288,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,4675.104705,NaN
603,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,56081.504073,NaN
889,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,67465.202576,NaN
1138,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,66697.446865,NaN
1418,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,37980.277154,NaN
1705,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,44842.076419,NaN
1977,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,38060.316661,NaN
2302,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,36887.598446,NaN
2512,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,38493.083105,NaN


In [15]:
parks_df = parks_df.drop_duplicates(subset = 'name', keep = 'first')  #dropping duplicate name and keeping first occurence
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,docjbpsuMVzYyF-8TVEEBQ,rockland-recreation-area-hendersonville,Rockland Recreation Area,https://s3-media1.fl.yelpcdn.com/bphoto/7ePq3z...,False,https://www.yelp.com/biz/rockland-recreation-a...,4,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.3009417057037, 'longitude': -8...",[],"{'address1': '5 Power Plant Rd', 'address2': '...",+16158224846,(615) 822-4846,40767.341539,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,UYEcvVgNhgkbUt4Z7p12WQ,cummins-falls-state-park-cookeville,Cummins Falls State Park,https://s3-media2.fl.yelpcdn.com/bphoto/SxT4ho...,False,https://www.yelp.com/biz/cummins-falls-state-p...,84,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.253729, 'longitude': -85.56541}",[],"{'address1': '390 Cummins Falls Ln', 'address2...",+19312687223,(931) 268-7223,67337.951515,NaN
2529,7GfuDk7KLOKorBO2qWi7KA,burgess-falls-state-park-sparta,Burgess Falls State Park,https://s3-media1.fl.yelpcdn.com/bphoto/tLPs6w...,False,https://www.yelp.com/biz/burgess-falls-state-p...,53,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.01357104300603, 'longitude': -...",[],"{'address1': '4000 Burgess Falls Dr', 'address...",+19314325312,(931) 432-5312,69624.185315,NaN
2535,HtWDageMy-cNd07cBWQSeQ,dogwood-park-cookeville,Dogwood park,https://s3-media2.fl.yelpcdn.com/bphoto/weRvOr...,False,https://www.yelp.com/biz/dogwood-park-cookevil...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.16433127651388, 'longitude': -...",[],"{'address1': '30 E Broad St', 'address2': '', ...",+19315204386,(931) 520-4386,72050.522374,NaN
2597,5K_LErcS6RruRR1o0iKIjQ,cane-creek-park-cookeville,Cane Creek Park,,False,https://www.yelp.com/biz/cane-creek-park-cooke...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1611979931865, 'longitude': -8...",[],"{'address1': '45 E Broad St', 'address2': None...",+19315269591,(931) 526-9591,68802.462858,NaN


In [16]:
parks_df = parks_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,docjbpsuMVzYyF-8TVEEBQ,rockland-recreation-area-hendersonville,Rockland Recreation Area,https://s3-media1.fl.yelpcdn.com/bphoto/7ePq3z...,False,https://www.yelp.com/biz/rockland-recreation-a...,4,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.3009417057037, 'longitude': -8...",[],"{'address1': '5 Power Plant Rd', 'address2': '...",+16158224846,(615) 822-4846,40767.341539,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,UYEcvVgNhgkbUt4Z7p12WQ,cummins-falls-state-park-cookeville,Cummins Falls State Park,https://s3-media2.fl.yelpcdn.com/bphoto/SxT4ho...,False,https://www.yelp.com/biz/cummins-falls-state-p...,84,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.253729, 'longitude': -85.56541}",[],"{'address1': '390 Cummins Falls Ln', 'address2...",+19312687223,(931) 268-7223,67337.951515,NaN
348,7GfuDk7KLOKorBO2qWi7KA,burgess-falls-state-park-sparta,Burgess Falls State Park,https://s3-media1.fl.yelpcdn.com/bphoto/tLPs6w...,False,https://www.yelp.com/biz/burgess-falls-state-p...,53,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.01357104300603, 'longitude': -...",[],"{'address1': '4000 Burgess Falls Dr', 'address...",+19314325312,(931) 432-5312,69624.185315,NaN
349,HtWDageMy-cNd07cBWQSeQ,dogwood-park-cookeville,Dogwood park,https://s3-media2.fl.yelpcdn.com/bphoto/weRvOr...,False,https://www.yelp.com/biz/dogwood-park-cookevil...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.16433127651388, 'longitude': -...",[],"{'address1': '30 E Broad St', 'address2': '', ...",+19315204386,(931) 520-4386,72050.522374,NaN
350,5K_LErcS6RruRR1o0iKIjQ,cane-creek-park-cookeville,Cane Creek Park,,False,https://www.yelp.com/biz/cane-creek-park-cooke...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1611979931865, 'longitude': -8...",[],"{'address1': '45 E Broad St', 'address2': None...",+19315269591,(931) 526-9591,68802.462858,NaN


In [17]:
parks_df[300:352]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
300,W2oo2INuua33jmtQB8O3Eg,farmington-confederate-memorial-park-lewisburg,Farmington Confederate Memorial Park,https://s3-media1.fl.yelpcdn.com/bphoto/_S9r0C...,False,https://www.yelp.com/biz/farmington-confederat...,1,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,"{'latitude': 35.4980995652881, 'longitude': -8...",[],"{'address1': '1066 Kenny Nelson Road', 'addres...",,,93404.103834,NaN
301,5pC3-MON_xZEVkYO9Rvy9A,almost-home-portable-buildings-dover,Almost Home Portable Buildings,https://s3-media1.fl.yelpcdn.com/bphoto/BIHbtG...,False,https://www.yelp.com/biz/almost-home-portable-...,1,"[{'alias': 'shedsandoutdoorstorage', 'title': ...",1.0,"{'latitude': 36.452235, 'longitude': -88.01926...",[],"{'address1': '3489 Donelson Pkwy', 'address2':...",+19316273180,(931) 627-3180,68263.556904,NaN
302,vhI8jnvZQMp9g-BbR7kSZQ,paris-landing-kentucky-lake-koa-buchanan,Paris Landing / Kentucky Lake KOA,https://s3-media2.fl.yelpcdn.com/bphoto/0SUBzf...,False,https://www.yelp.com/biz/paris-landing-kentuck...,10,"[{'alias': 'campgrounds', 'title': 'Campground...",4.5,"{'latitude': 36.413924, 'longitude': -88.131851}",[],"{'address1': '6290 E Antioch Rd', 'address2': ...",+17316426895,(731) 642-6895,75508.746890,NaN
303,3cHRFk0kQesNu8ZpexBFwg,stillhouse-hollow-falls-trail-mount-pleasant,Stillhouse Hollow Falls Trail,https://s3-media1.fl.yelpcdn.com/bphoto/5RB3Sb...,False,https://www.yelp.com/biz/stillhouse-hollow-fal...,10,"[{'alias': 'hiking', 'title': 'Hiking'}]",4.5,"{'latitude': 35.4632773446791, 'longitude': -8...",[],"{'address1': 'Stillhouse Hollow Falls Trl', 'a...",,,26138.702587,NaN
304,KIt7nfI2FBqk3FtRjIoGag,david-crockett-state-park-lawrenceburg-5,David Crockett State Park,https://s3-media4.fl.yelpcdn.com/bphoto/rESJAc...,False,https://www.yelp.com/biz/david-crockett-state-...,12,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.0,"{'latitude': 35.2428681949774, 'longitude': -8...",[],"{'address1': '1400 West Gaines', 'address2': '...",+19319032323,(931) 903-2323,50301.161139,NaN
305,7okvnrjCaH5Xp9VPFcvrHg,stone-bridge-memorial-park-fayetteville,Stone Bridge Memorial Park,https://s3-media2.fl.yelpcdn.com/bphoto/-X373V...,False,https://www.yelp.com/biz/stone-bridge-memorial...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 35.144807, 'longitude': -86.568592}",[],"{'address1': 'Eufala St', 'address2': '', 'add...",,,70334.415134,NaN
306,SkGinj_BiRFbY6QQ-Ln9rA,rock-creek-park-lewisburg,Rock Creek Park,https://s3-media2.fl.yelpcdn.com/bphoto/ZVVBW0...,False,https://www.yelp.com/biz/rock-creek-park-lewis...,1,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 35.453433112927, 'longitude': -86...",[],"{'address1': '101 Old Farmington Rd', 'address...",+19313591544,(931) 359-1544,32122.512354,NaN
307,zT6ayvRBuGF7OYPQA4dDYg,tims-ford-state-park-winchester-2,Tims Ford State Park,https://s3-media1.fl.yelpcdn.com/bphoto/jJKA65...,False,https://www.yelp.com/biz/tims-ford-state-park-...,13,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 35.219827, 'longitude': -86.255447}",[],"{'address1': '570 Tims Ford Dr', 'address2': N...",+19319683536,(931) 968-3536,86492.203190,NaN
308,JTKl-jQK76k5Cmdfd0AknQ,lewisburg-parks-rec-and-fitness-lewisburg,Lewisburg Parks Rec and Fitness,https://s3-media2.fl.yelpcdn.com/bphoto/r-GNkl...,False,https://www.yelp.com/biz/lewisburg-parks-rec-a...,2,"[{'alias': 'recreation', 'title': 'Recreation ...",4.5,"{'latitude': 35.4492545750614, 'longitude': -8...",[],"{'address1': '1551 Mooresville Hwy', 'address2...",+19313592482,(931) 359-2482,30481.554137,NaN
309,W8xHRMRMg8CqPh_IA6Eufg,mitzi-sweet-dog-park-lawrenceburg,Mitzi Sweet Dog Park,https://s3-media1.fl.yelpcdn.com/bphoto/D5wQt4...,False,https://www.yelp.com/biz/mitzi-sweet-dog-park-...,2,"[{'alias': 'dog_parks', 'title': 'Dog Parks'}]",4.5,"{'latitude': 35.2354

In [18]:
parks_df = parks_df.drop([185,270,296,297,298,300,315,341])
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,docjbpsuMVzYyF-8TVEEBQ,rockland-recreation-area-hendersonville,Rockland Recreation Area,https://s3-media1.fl.yelpcdn.com/bphoto/7ePq3z...,False,https://www.yelp.com/biz/rockland-recreation-a...,4,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.3009417057037, 'longitude': -8...",[],"{'address1': '5 Power Plant Rd', 'address2': '...",+16158224846,(615) 822-4846,40767.341539,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,UYEcvVgNhgkbUt4Z7p12WQ,cummins-falls-state-park-cookeville,Cummins Falls State Park,https://s3-media2.fl.yelpcdn.com/bphoto/SxT4ho...,False,https://www.yelp.com/biz/cummins-falls-state-p...,84,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.253729, 'longitude': -85.56541}",[],"{'address1': '390 Cummins Falls Ln', 'address2...",+19312687223,(931) 268-7223,67337.951515,NaN
348,7GfuDk7KLOKorBO2qWi7KA,burgess-falls-state-park-sparta,Burgess Falls State Park,https://s3-media1.fl.yelpcdn.com/bphoto/tLPs6w...,False,https://www.yelp.com/biz/burgess-falls-state-p...,53,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.01357104300603, 'longitude': -...",[],"{'address1': '4000 Burgess Falls Dr', 'address...",+19314325312,(931) 432-5312,69624.185315,NaN
349,HtWDageMy-cNd07cBWQSeQ,dogwood-park-cookeville,Dogwood park,https://s3-media2.fl.yelpcdn.com/bphoto/weRvOr...,False,https://www.yelp.com/biz/dogwood-park-cookevil...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.16433127651388, 'longitude': -...",[],"{'address1': '30 E Broad St', 'address2': '', ...",+19315204386,(931) 520-4386,72050.522374,NaN
350,5K_LErcS6RruRR1o0iKIjQ,cane-creek-park-cookeville,Cane Creek Park,,False,https://www.yelp.com/biz/cane-creek-park-cooke...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1611979931865, 'longitude': -8...",[],"{'address1': '45 E Broad St', 'address2': None...",+19315269591,(931) 526-9591,68802.462858,NaN


In [19]:
#now will repeat similar process to gather other activites for families in the 10 counties

In [20]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
museums = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'museums', location = location, search_limit = 20, offset = x)
        museums.append(response)
        print(len(museums))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [21]:
museums_df = pd.DataFrame()

for museum in range(len(museums)):
    df = pd.DataFrame(museums[museum]['businesses'])
    museums_df = museums_df.append(df)
    print(museums_df.shape)

(20, 15)
(40, 16)
(60, 16)
(80, 16)
(100, 16)
(120, 16)
(140, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(162, 16)
(182, 16)
(202, 16)
(222, 16)
(242, 16)
(262, 16)
(282, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(303, 16)
(323, 16)
(343, 16)
(363, 16)
(383, 16)
(403, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(435, 16)
(455, 16)
(475, 16)
(495, 16)
(515, 16)
(535, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(567

In [22]:
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
3,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,Xnp_mwco11OL_mAp4IBwaA,picture-frame-warehouse-nashville-4,Picture Frame Warehouse,https://s3-media1.fl.yelpcdn.com/bphoto/5JUZiz...,False,https://www.yelp.com/biz/picture-frame-warehou...,13,"[{'alias': 'framing', 'title': 'Framing'}, {'a...",3.5,"{'latitude': 36.09439865389617, 'longitude': -...",[],"{'address1': '4320 Kenilwood Dr', 'address2': ...",+16153331161,(615) 333-1161,41482.530900,$$$
7,_9Z0aVjzCXR7ASwVe5xcSw,cma-fest-nashville-4,CMA Fest,https://s3-media4.fl.yelpcdn.com/bphoto/dVsmI3...,False,https://www.yelp.com/biz/cma-fest-nashville-4?...,29,"[{'alias': 'festivals', 'title': 'Festivals'}]",4.0,"{'latitude': 36.16558525534611, 'longitude': -...",[],"{'address1': '1 Titans Way', 'address2': '', '...",+16152442840,(615) 244-2840,41706.774060,NaN
8,h874qqAB4vMhUh-xz8RtiA,southern-ground-music-and-food-festival-nashville,Southern Ground Music & Food Festival,https://s3-media4.fl.yelpcdn.com/bphoto/Jfsjs6...,False,https://www.yelp.com/biz/southern-ground-music...,2,"[{'alias': 'festivals', 'title': 'Festivals'}]",4.5,"{'latitude': 36.1602222, 'longitude': -86.7730...",[],"{'address1': '200 1st Ave S', 'address2': '', ...",,,41952.718048,NaN
9,sO1OD0SZkXXPtIXj2cS4oA,the-stage-on-broadway-nashville-2,The Stage on Broadway,https://s3-media2.fl.yelpcdn.com/bphoto/kdPBlz...,False,https://www.yelp.com/biz/the-stage-on-broadway...,423,"[{'alias': 'musicvenues', 'title': 'Music Venu...",4.0,"{'latitude': 36.16111, 'longitude': -86.777898}",[],"{'address1': '412 Broadway', 'address2': '', '...",+16157260504,(615) 726-0504,42385.292016,$$


In [23]:
museums_df = museums_df.reset_index(drop=True)   
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
3,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,Xnp_mwco11OL_mAp4IBwaA,picture-frame-warehouse-nashville-4,Picture Frame Warehouse,https://s3-media1.fl.yelpcdn.com/bphoto/5JUZiz...,False,https://www.yelp.com/biz/picture-frame-warehou...,13,"[{'alias': 'framing', 'title': 'Framing'}, {'a...",3.5,"{'latitude': 36.09439865389617, 'longitude': -...",[],"{'address1': '4320 Kenilwood Dr', 'address2': ...",+16153331161,(615) 333-1161,41482.530900,$$$
1354,_9Z0aVjzCXR7ASwVe5xcSw,cma-fest-nashville-4,CMA Fest,https://s3-media4.fl.yelpcdn.com/bphoto/dVsmI3...,False,https://www.yelp.com/biz/cma-fest-nashville-4?...,29,"[{'alias': 'festivals', 'title': 'Festivals'}]",4.0,"{'latitude': 36.16558525534611, 'longitude': -...",[],"{'address1': '1 Titans Way', 'address2': '', '...",+16152442840,(615) 244-2840,41706.774060,NaN
1355,h874qqAB4vMhUh-xz8RtiA,southern-ground-music-and-food-festival-nashville,Southern Ground Music & Food Festival,https://s3-media4.fl.yelpcdn.com/bphoto/Jfsjs6...,False,https://www.yelp.com/biz/southern-ground-music...,2,"[{'alias': 'festivals', 'title': 'Festivals'}]",4.5,"{'latitude': 36.1602222, 'longitude': -86.7730...",[],"{'address1': '200 1st Ave S', 'address2': '', ...",,,41952.718048,NaN
1356,sO1OD0SZkXXPtIXj2cS4oA,the-stage-on-broadway-nashville-2,The Stage on Broadway,https://s3-media2.fl.yelpcdn.com/bphoto/kdPBlz...,False,https://www.yelp.com/biz/the-stage-on-broadway...,423,"[{'alias': 'musicvenues', 'title': 'Music Venu...",4.0,"{'latitude': 36.16111, 'longitude': -86.777898}",[],"{'address1': '412 Broadway', 'address2': '', '...",+16157260504,(615) 726-0504,42385.292016,$$


In [24]:
museums_df[museums_df.duplicated(subset = 'name', keep = 'first')]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
142,mnwTbHyIgpMvOG9705zqWA,country-music-hall-of-fame-and-museum-nashville-3,Country Music Hall of Fame and Museum,https://s3-media3.fl.yelpcdn.com/bphoto/lr2eAA...,False,https://www.yelp.com/biz/country-music-hall-of...,688,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.158266, 'longitude': -86.776126}",[],"{'address1': '222 Rep. John Lewis Way S', 'add...",+16154162001,(615) 416-2001,3153.820124,NaN
143,z2FRDJSxRR3mmC9syR25xA,the-parthenon-nashville,The Parthenon,https://s3-media2.fl.yelpcdn.com/bphoto/juyVpf...,False,https://www.yelp.com/biz/the-parthenon-nashvil...,475,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",4.0,"{'latitude': 36.14967469964605, 'longitude': -...",[],"{'address1': '2500 West End Ave', 'address2': ...",+16158628431,(615) 862-8431,4830.218646,NaN
144,jSYu2apG1uYq3eNzSA-OHw,johnny-cash-museum-nashville-4,Johnny Cash Museum,https://s3-media2.fl.yelpcdn.com/bphoto/qkanvY...,False,https://www.yelp.com/biz/johnny-cash-museum-na...,433,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.16084, 'longitude': -86.77586}",[],"{'address1': '119 3rd Ave S', 'address2': '', ...",+16152561777,(615) 256-1777,2877.143845,NaN
145,z87SlFdP6vZCjfjPkI-edQ,national-museum-of-african-american-music-nash...,National Museum of African American Music,https://s3-media2.fl.yelpcdn.com/bphoto/ZfjG4_...,False,https://www.yelp.com/biz/national-museum-of-af...,72,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16053057546768, 'longitude': -...",[],"{'address1': '510 Broadway', 'address2': '', '...",+16153018724,(615) 301-8724,2862.000472,NaN
146,gNZC4qA7eMLQpUMeCYPAoA,frist-art-museum-nashville,Frist Art Museum,https://s3-media2.fl.yelpcdn.com/bphoto/EOAggj...,False,https://www.yelp.com/biz/frist-art-museum-nash...,230,"[{'alias': 'artmuseums', 'title': 'Art Museums'}]",4.0,"{'latitude': 36.157657, 'longitude': -86.783619}",[],"{'address1': '919 Broadway', 'address2': '', '...",+16152443340,(615) 244-3340,3146.369231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,Xnp_mwco11OL_mAp4IBwaA,picture-frame-warehouse-nashville-4,Picture Frame Warehouse,https://s3-media1.fl.yelpcdn.com/bphoto/5JUZiz...,False,https://www.yelp.com/biz/picture-frame-warehou...,13,"[{'alias': 'framing', 'title': 'Framing'}, {'a...",3.5,"{'latitude': 36.09439865389617, 'longitude': -...",[],"{'address1': '4320 Kenilwood Dr', 'address2': ...",+16153331161,(615) 333-1161,41482.530900,$$$
1354,_9Z0aVjzCXR7ASwVe5xcSw,cma-fest-nashville-4,CMA Fest,https://s3-media4.fl.yelpcdn.com/bphoto/dVsmI3...,False,https://www.yelp.com/biz/cma-fest-nashville-4?...,29,"[{'alias': 'festivals', 'title': 'Festivals'}]",4.0,"{'latitude': 36.16558525534611, 'longitude': -...",[],"{'address1': '1 Titans Way', 'address2': '', '...",+16152442840,(615) 244-2840,41706.774060,NaN
1355,h874qqAB4vMhUh-xz8RtiA,southern-ground-music-and-food-festival-nashville,Southern Ground Music & Food Festival,https://s3-media4.fl.yelpcdn.com/bphoto/Jfsjs6...,False,https://www.yelp.com/biz/southern-ground-music...,2,"[{'alias': 'festivals', 'title': 'Festivals'}]",4.5,"{'latitude': 36.1602222, 'longitude': -86.7730...",[],"{'address1': '200 1st Ave S', 'address2': '', ...",,,41952.718048,NaN
1356,sO1OD0SZkXXPtIXj2cS4oA,the-stage-on-broadway-nashville-2,The Stage on Broadway,https://s3-media2.fl.yelpcdn.com/bphoto/kdPBlz...,False,https://www.yelp.com/biz/the-stage-on-broadway...,423,"[{'alias': 'musicvenues', 'title': 'Music Venu...",4.0,"{'latitude': 36.16111, 'longitude': -86.777898}",[],"{'address1': '412 Broadway', 'address2': '', '...",+16157260504,(615) 726-0504,42385.292016,$$


In [25]:
museums_df = museums_df.drop_duplicates(subset = 'name', keep = 'first')  
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
3,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,N6BB-kevywd7Zt07E0RXEA,the-smoke-house-lodge-and-cabins-monteagle,The Smoke House Lodge & Cabins,https://s3-media2.fl.yelpcdn.com/bphoto/_FN36a...,False,https://www.yelp.com/biz/the-smoke-house-lodge...,111,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",3.0,"{'latitude': 35.235441, 'longitude': -85.852699}",[],"{'address1': '844 W Main St', 'address2': '', ...",+19319242091,(931) 924-2091,86367.245308,$$
960,3B7sd1IplPrxdEkHNOC8ag,south-central-kentucky-cultural-center-glasgow,South Central Kentucky Cultural Center,https://s3-media3.fl.yelpcdn.com/bphoto/XzXG94...,False,https://www.yelp.com/biz/south-central-kentuck...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.996957, 'longitude': -85.912694}",[],"{'address1': '200 W Water St', 'address2': '',...",+12706519792,(270) 651-9792,79276.910092,NaN
1235,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN
1239,BGhgxzyl4ECtYBzlU8iG3g,cookeville-childrens-museum-cookeville,Cookeville Childrens Museum,https://s3-media4.fl.yelpcdn.com/bphoto/zW3gER...,False,https://www.yelp.com/biz/cookeville-childrens-...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16605, 'longitude': -85.5072403}",[],"{'address1': '36 W 2nd St', 'address2': None, ...",+19319797529,(931) 979-7529,71703.003792,NaN


In [26]:
museums_df = museums_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
3,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,N6BB-kevywd7Zt07E0RXEA,the-smoke-house-lodge-and-cabins-monteagle,The Smoke House Lodge & Cabins,https://s3-media2.fl.yelpcdn.com/bphoto/_FN36a...,False,https://www.yelp.com/biz/the-smoke-house-lodge...,111,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",3.0,"{'latitude': 35.235441, 'longitude': -85.852699}",[],"{'address1': '844 W Main St', 'address2': '', ...",+19319242091,(931) 924-2091,86367.245308,$$
174,3B7sd1IplPrxdEkHNOC8ag,south-central-kentucky-cultural-center-glasgow,South Central Kentucky Cultural Center,https://s3-media3.fl.yelpcdn.com/bphoto/XzXG94...,False,https://www.yelp.com/biz/south-central-kentuck...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.996957, 'longitude': -85.912694}",[],"{'address1': '200 W Water St', 'address2': '',...",+12706519792,(270) 651-9792,79276.910092,NaN
175,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN
176,BGhgxzyl4ECtYBzlU8iG3g,cookeville-childrens-museum-cookeville,Cookeville Childrens Museum,https://s3-media4.fl.yelpcdn.com/bphoto/zW3gER...,False,https://www.yelp.com/biz/cookeville-childrens-...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16605, 'longitude': -85.5072403}",[],"{'address1': '36 W 2nd St', 'address2': None, ...",+19319797529,(931) 979-7529,71703.003792,NaN


In [27]:
museums_df = museums_df.drop([18,21,35,45,49,54,62,65,69,83,85,86,89,96,97,102,104,105,106,107,108,109,110,111,112,113,115,117,120,121,124,126,127,128,130,131,132,133,134,135,138,139,140,141,142,156,164,165,166,171,172,173])
museums_df
#removing all rows that are not museums

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
3,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,dtdOcQ-v92f2tuZvvi_paQ,monty-wanamakers-southern-museum-and-gallery-m...,Monty Wanamaker's Southern Museum & Gallery,https://s3-media1.fl.yelpcdn.com/bphoto/-YcjT5...,False,https://www.yelp.com/biz/monty-wanamakers-sout...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 35.6807518005371, 'longitude': -8...",[],"{'address1': '210 E Main St', 'address2': '', ...",+19315078102,(931) 507-8102,61908.000011,NaN
174,3B7sd1IplPrxdEkHNOC8ag,south-central-kentucky-cultural-center-glasgow,South Central Kentucky Cultural Center,https://s3-media3.fl.yelpcdn.com/bphoto/XzXG94...,False,https://www.yelp.com/biz/south-central-kentuck...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.996957, 'longitude': -85.912694}",[],"{'address1': '200 W Water St', 'address2': '',...",+12706519792,(270) 651-9792,79276.910092,NaN
175,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN
176,BGhgxzyl4ECtYBzlU8iG3g,cookeville-childrens-museum-cookeville,Cookeville Childrens Museum,https://s3-media4.fl.yelpcdn.com/bphoto/zW3gER...,False,https://www.yelp.com/biz/cookeville-childrens-...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16605, 'longitude': -85.5072403}",[],"{'address1': '36 W 2nd St', 'address2': None, ...",+19319797529,(931) 979-7529,71703.003792,NaN


In [28]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
landmarks = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'landmarks', location = location, search_limit = 20, offset = x)
        landmarks.append(response)
        print(len(landmarks))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [29]:
landmarks_df = pd.DataFrame()

for landmark in range(len(landmarks)):
    df = pd.DataFrame(landmarks[landmark]['businesses'])
    landmarks_df = landmarks_df.append(df)
    print(landmarks_df.shape)

(20, 16)
(40, 16)
(60, 16)
(80, 16)
(100, 16)
(120, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(138, 16)
(158, 16)
(178, 16)
(198, 16)
(218, 16)
(238, 16)
(258, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(272, 16)
(292, 16)
(312, 16)
(332, 16)
(352, 16)
(372, 16)
(392, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(399, 16)
(419, 16)
(439, 16)
(459, 16)
(479, 16)
(499, 16)
(519, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(527, 16)
(547

In [30]:
landmarks_df = landmarks_df.reset_index(drop=True)   
landmarks_df

#seeing that most rows are not landmarks and have already been grabbed in museum search

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,3eEFi_7n893TAveP-4dJNQ,confederate-soldiers-monument-franklin,Confederate Soldiers Monument,https://s3-media3.fl.yelpcdn.com/bphoto/fnZrNx...,False,https://www.yelp.com/biz/confederate-soldiers-...,4,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",4.5,"{'latitude': 35.9251706043456, 'longitude': -8...",[],"{'address1': '310 Main St', 'address2': '', 'a...",,,41327.767958,NaN
2,ARKtuM_CFDs427PjEXKwQw,carnton-franklin,Carnton,https://s3-media4.fl.yelpcdn.com/bphoto/gwsqRP...,False,https://www.yelp.com/biz/carnton-franklin?adju...,88,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",4.5,"{'latitude': 35.903964588891, 'longitude': -86...",[],"{'address1': '1345 Eastern Flank Cir', 'addres...",+16157940903,(615) 794-0903,43855.047113,NaN
3,FMOf8g3gk5nJF3Dd6lWSCQ,tennesseetriumph-womens-suffrage-monument-clar...,TennesseeTriumph Women's Suffrage Monument,https://s3-media3.fl.yelpcdn.com/bphoto/1bVLGC...,False,https://www.yelp.com/biz/tennesseetriumph-wome...,1,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,"{'latitude': 36.5272571, 'longitude': -87.361608}",[],"{'address1': '50 Public Square', 'address2': '...",,,39031.794405,NaN
4,z2FRDJSxRR3mmC9syR25xA,the-parthenon-nashville,The Parthenon,https://s3-media2.fl.yelpcdn.com/bphoto/juyVpf...,False,https://www.yelp.com/biz/the-parthenon-nashvil...,475,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",4.0,"{'latitude': 36.14967469964605, 'longitude': -...",[],"{'address1': '2500 West End Ave', 'address2': ...",+16158628431,(615) 862-8431,27689.774663,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,3X9LroR0M2Hp7ULP7wlDgA,stix-by-christian-moeller-nashville,Stix by Christian Moeller,https://s3-media2.fl.yelpcdn.com/bphoto/r430Xg...,False,https://www.yelp.com/biz/stix-by-christian-moe...,3,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",3.5,"{'latitude': 36.1547562192515, 'longitude': -8...",[],"{'address1': None, 'address2': None, 'address3...",,,42520.833863,NaN
1309,VK15e4gDR4GO4jOmbrNOzA,sampson-w-keeble-historical-marker-nashville,Sampson W. Keeble Historical Marker,https://s3-media1.fl.yelpcdn.com/bphoto/YsdHd-...,False,https://www.yelp.com/biz/sampson-w-keeble-hist...,1,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",1.0,"{'latitude': 36.1623268127441, 'longitude': -8...",[],"{'address1': '108 2nd Ave N', 'address2': '', ...",,,42131.727073,NaN
1310,gfw5Qzb51ewVtE_g5oQH_Q,nashville-gov-nashville,Nashville.Gov,https://s3-media2.fl.yelpcdn.com/bphoto/Nnspqg...,False,https://www.yelp.com/biz/nashville-gov-nashvil...,3,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",3.5,"{'latitude': 36.1626638, 'longitude': -86.7812...",[],"{'address1': '100 Metro Cthouse', 'address2': ...",+16158626000,(615) 862-6000,42685.794144,NaN
1311,77IcM0Oa6TymDNWeZnkBbA,birth-of-bluegrass-historical-marker-nashville,Birth of Bluegrass Historical Marker,https://s3-media4.fl.yelpcdn.com/bphoto/C1x5tt...,False,https://www.yelp.com/biz/birth-of-bluegrass-hi...,1,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",3.0,"{'latitude': 36.16119, 'longitude': -86.778717}",[],"{'address1': '116 5th Ave N', 'address2': '', ...",,,42436.950153,NaN


In [31]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
libraries = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'libraries', location = location, search_limit = 20, offset = x)
        libraries.append(response)
        print(len(libraries))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [32]:
libraries_df = pd.DataFrame()

for library in range(len(libraries)):
    df = pd.DataFrame(libraries[library]['businesses'])
    libraries_df = libraries_df.append(df)
    print(libraries_df.shape)

(20, 15)
(40, 15)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(75, 16)
(95, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(129, 16)
(149, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(175, 16)
(195, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(221, 16)
(241, 16)
(245, 1

In [33]:
libraries_df = libraries_df.drop_duplicates(subset = 'name', keep = 'first')  
libraries_df = libraries_df.reset_index(drop=True)  
libraries_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,cjIbKTknnvvw8E7FO6SCzw,nashville-public-library-nashville-2,Nashville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/oEwtcu...,False,https://www.yelp.com/biz/nashville-public-libr...,87,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.161758, 'longitude': -86.781608}",[],"{'address1': '615 Church St', 'address2': '', ...",+16158625800,(615) 862-5800,29867.224261,NaN
1,VCDIfeiT3xzWR--eyJk8_g,nashville-public-library-green-hills-branch-na...,Nashville Public Library - Green Hills Branch,https://s3-media1.fl.yelpcdn.com/bphoto/dTb9Zg...,False,https://www.yelp.com/biz/nashville-public-libr...,20,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 36.109909, 'longitude': -86.809107}",[],"{'address1': '3701 Benham Ave', 'address2': ''...",+16158625863,(615) 862-5863,30046.783562,NaN
2,Cs7hHsWMLvMxtxwDsRJCGw,nashville-public-library-bellevue-branch-nashv...,Nashville Public Library - Bellevue Branch,https://s3-media2.fl.yelpcdn.com/bphoto/tVkhdT...,False,https://www.yelp.com/biz/nashville-public-libr...,12,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.0691351443529, 'longitude': -8...",[],"{'address1': '720 Baugh Rd', 'address2': '', '...",+16158625854,(615) 862-5854,24538.844400,NaN
3,cAIIsVrYIlBm8M4HHKqOrA,clarksville-montgomery-county-public-library-c...,Clarksville Montgomery County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/Vj0Epj...,False,https://www.yelp.com/biz/clarksville-montgomer...,16,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.521023, 'longitude': -87.340736}",[],"{'address1': '350 Pageant Ln', 'address2': 'St...",+19316488826,(931) 648-8826,37393.358988,NaN
4,71-dWjMS7aBUc0oG4wdsww,spring-hill-library-spring-hill,Spring Hill Library,https://s3-media3.fl.yelpcdn.com/bphoto/4ER2qm...,False,https://www.yelp.com/biz/spring-hill-library-s...,2,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.74972, 'longitude': -86.938}",[],"{'address1': '144 Kedron Pkwy', 'address2': ''...",+19314862932,(931) 486-2932,57236.245050,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,vqsCccA6jfcH34muTD1uHA,city-of-bowling-green-bowling-green-2,City of Bowling Green,https://s3-media4.fl.yelpcdn.com/bphoto/t5PJsg...,False,https://www.yelp.com/biz/city-of-bowling-green...,1,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,"{'latitude': 36.9924079, 'longitude': -86.4438...",[],"{'address1': '1001 College St', 'address2': ''...",+12703933000,(270) 393-3000,66014.808955,NaN
61,gjibOZdKcD1t8cFRg4Ak5A,coffee-county-lannom-memorial-library-tullahoma,Coffee County Lannom Memorial Library,,False,https://www.yelp.com/biz/coffee-county-lannom-...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.3608638, 'longitude': -86.2192...",[],"{'address1': '312 North Collins St', 'address2...",+19314552460,(931) 455-2460,58364.385353,NaN
62,HoLD1c5ivg_zMxGjHYCxgA,manchester-branch-library-manchester,Manchester Branch Library,,False,https://www.yelp.com/biz/manchester-branch-lib...,3,"[{'alias': 'libraries', 'title': 'Libraries'}]",2.0,"{'latitude': 35.47384, 'longitude': -86.07763}",[],"{'address1': '1005 Hillsboro Blvd', 'address2'...",+19317235143,(931) 723-5143,52928.010680,NaN
63,YwU6Rrrx5JgU2w8UVjeFyQ,morrison-library-morrison,Morrison Library,,False,https://www.yelp.com/biz/morrison-library-morr...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",1.0,"{'latitude': 35.60276, 'longitude': -85.91965}",[],"{'address1': '100 W Maple St', 'address2': '',...",+19316352060,(931) 635-2060,53513.114447,NaN


In [34]:
libraries_df = libraries_df.drop([])
libraries_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,cjIbKTknnvvw8E7FO6SCzw,nashville-public-library-nashville-2,Nashville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/oEwtcu...,False,https://www.yelp.com/biz/nashville-public-libr...,87,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.161758, 'longitude': -86.781608}",[],"{'address1': '615 Church St', 'address2': '', ...",+16158625800,(615) 862-5800,29867.224261,NaN
1,VCDIfeiT3xzWR--eyJk8_g,nashville-public-library-green-hills-branch-na...,Nashville Public Library - Green Hills Branch,https://s3-media1.fl.yelpcdn.com/bphoto/dTb9Zg...,False,https://www.yelp.com/biz/nashville-public-libr...,20,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 36.109909, 'longitude': -86.809107}",[],"{'address1': '3701 Benham Ave', 'address2': ''...",+16158625863,(615) 862-5863,30046.783562,NaN
2,Cs7hHsWMLvMxtxwDsRJCGw,nashville-public-library-bellevue-branch-nashv...,Nashville Public Library - Bellevue Branch,https://s3-media2.fl.yelpcdn.com/bphoto/tVkhdT...,False,https://www.yelp.com/biz/nashville-public-libr...,12,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.0691351443529, 'longitude': -8...",[],"{'address1': '720 Baugh Rd', 'address2': '', '...",+16158625854,(615) 862-5854,24538.844400,NaN
3,cAIIsVrYIlBm8M4HHKqOrA,clarksville-montgomery-county-public-library-c...,Clarksville Montgomery County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/Vj0Epj...,False,https://www.yelp.com/biz/clarksville-montgomer...,16,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.521023, 'longitude': -87.340736}",[],"{'address1': '350 Pageant Ln', 'address2': 'St...",+19316488826,(931) 648-8826,37393.358988,NaN
4,71-dWjMS7aBUc0oG4wdsww,spring-hill-library-spring-hill,Spring Hill Library,https://s3-media3.fl.yelpcdn.com/bphoto/4ER2qm...,False,https://www.yelp.com/biz/spring-hill-library-s...,2,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.74972, 'longitude': -86.938}",[],"{'address1': '144 Kedron Pkwy', 'address2': ''...",+19314862932,(931) 486-2932,57236.245050,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,vqsCccA6jfcH34muTD1uHA,city-of-bowling-green-bowling-green-2,City of Bowling Green,https://s3-media4.fl.yelpcdn.com/bphoto/t5PJsg...,False,https://www.yelp.com/biz/city-of-bowling-green...,1,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,"{'latitude': 36.9924079, 'longitude': -86.4438...",[],"{'address1': '1001 College St', 'address2': ''...",+12703933000,(270) 393-3000,66014.808955,NaN
61,gjibOZdKcD1t8cFRg4Ak5A,coffee-county-lannom-memorial-library-tullahoma,Coffee County Lannom Memorial Library,,False,https://www.yelp.com/biz/coffee-county-lannom-...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.3608638, 'longitude': -86.2192...",[],"{'address1': '312 North Collins St', 'address2...",+19314552460,(931) 455-2460,58364.385353,NaN
62,HoLD1c5ivg_zMxGjHYCxgA,manchester-branch-library-manchester,Manchester Branch Library,,False,https://www.yelp.com/biz/manchester-branch-lib...,3,"[{'alias': 'libraries', 'title': 'Libraries'}]",2.0,"{'latitude': 35.47384, 'longitude': -86.07763}",[],"{'address1': '1005 Hillsboro Blvd', 'address2'...",+19317235143,(931) 723-5143,52928.010680,NaN
63,YwU6Rrrx5JgU2w8UVjeFyQ,morrison-library-morrison,Morrison Library,,False,https://www.yelp.com/biz/morrison-library-morr...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",1.0,"{'latitude': 35.60276, 'longitude': -85.91965}",[],"{'address1': '100 W Maple St', 'address2': '',...",+19316352060,(931) 635-2060,53513.114447,NaN


In [35]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
arcades = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'arcades', location = location, search_limit = 20, offset = x)
        arcades.append(response)
        print(len(arcades))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [36]:
arcades_df = pd.DataFrame()

for arcade in range(len(arcades)):
    df = pd.DataFrame(arcades[arcade]['businesses'])
    arcades_df = arcades_df.append(df)
    print(arcades_df.shape)

(20, 16)
(40, 16)
(60, 16)
(80, 16)
(100, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(127, 16)
(147, 16)
(167, 16)
(187, 16)
(207, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(233, 16)
(253, 16)
(273, 16)
(293, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(319, 16)
(339, 16)
(359, 16)
(379, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(408

In [37]:
arcades_df = arcades_df.drop_duplicates(subset = 'name', keep = 'first')  
arcades_df = arcades_df.reset_index(drop=True)  
arcades_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,UiURMNx6i-qeBWvwcIKPgQ,joyfull-arcade-spring-hill-2,Joyfull Arcade,https://s3-media2.fl.yelpcdn.com/bphoto/z5EZ5s...,False,https://www.yelp.com/biz/joyfull-arcade-spring...,1,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.77542748017264, 'longitude': -...",[],"{'address1': '2001 Campbell Station Pkwy', 'ad...",+16156141184,(615) 614-1184,55092.784846,NaN
1,8mEcWgU7q6lWchl-aJy_dw,the-city-forum-clarksville,The City Forum,https://s3-media3.fl.yelpcdn.com/bphoto/nLNur6...,False,https://www.yelp.com/biz/the-city-forum-clarks...,56,"[{'alias': 'mini_golf', 'title': 'Mini Golf'},...",4.0,"{'latitude': 36.511204, 'longitude': -87.2745814}",[],"{'address1': '2231-T Madison St', 'address2': ...",+19319195051,(931) 919-5051,33312.136770,NaN
2,_qPvx7G1-Q5I8JYuDTXlPQ,warrior-zone-fort-campbell,Warrior Zone,https://s3-media3.fl.yelpcdn.com/bphoto/KCnYC6...,False,https://www.yelp.com/biz/warrior-zone-fort-cam...,1,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",4.0,"{'latitude': 36.66417655503705, 'longitude': -...",[],"{'address1': '3910 Indiana Ave', 'address2': '...",+12704610683,(270) 461-0683,56965.972820,NaN
3,3MuUUkRYnx56VAYyENvQDA,up-down-nashville-nashville-2,Up-Down Nashville,https://s3-media3.fl.yelpcdn.com/bphoto/JuPSXB...,False,https://www.yelp.com/biz/up-down-nashville-nas...,23,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 36.1759, 'longitude': -86.7546}","[pickup, delivery]","{'address1': '927 Woodland St', 'address2': No...",+16152263747,(615) 226-3747,31732.220217,$
4,1NYbb6BzNjEyUwFKc9UfZw,game-terminal-nashville,Game Terminal,https://s3-media3.fl.yelpcdn.com/bphoto/PO3erT...,False,https://www.yelp.com/biz/game-terminal-nashvil...,62,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",4.0,"{'latitude': 36.131339492100224, 'longitude': ...",[],"{'address1': '201 Terminal Ct', 'address2': ''...",+16156102460,(615) 610-2460,34131.360677,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,ztQK4EfzwMbj3zzceiJxAA,tullahoma-bowling-lanes-tullahoma,Tullahoma Bowling Lanes,https://s3-media2.fl.yelpcdn.com/bphoto/iXGjte...,False,https://www.yelp.com/biz/tullahoma-bowling-lan...,6,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",2.5,"{'latitude': 35.3563, 'longitude': -86.19797}",[],"{'address1': '900 S Anderson St', 'address2': ...",+19314551947,(931) 455-1947,59492.132681,NaN
107,tHVupO1RIdd-DjZ-BbytWQ,ralphies-fun-center-glasgow,Ralphie's Fun Center,https://s3-media4.fl.yelpcdn.com/bphoto/yoZf_k...,False,https://www.yelp.com/biz/ralphies-fun-center-g...,9,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",4.5,"{'latitude': 37.0086209, 'longitude': -85.9189...",[],"{'address1': '702 Happy Valley Rd', 'address2'...",+12706294263,(270) 629-4263,79927.217044,NaN
108,rvimEgjXRq38ACp1wFXi_g,paradise-games-and-arcade-cookeville,Paradise Games and Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/dYpjpD...,False,https://www.yelp.com/biz/paradise-games-and-ar...,1,"[{'alias': 'videogamestores', 'title': 'Video ...",5.0,"{'latitude': 36.16233218340348, 'longitude': -...",[],"{'address1': '8 S Washington Ave', 'address2':...",+19312149263,(931) 214-9263,72450.315101,NaN
109,Jdx6joQwAE_byUINnYl1Jg,the-sk8-cookeville-2,The Sk8,https://s3-media1.fl.yelpcdn.com/bphoto/eH_2Mt...,False,https://www.yelp.com/biz/the-sk8-cookeville-2?...,4,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.5,"{'latitude': 36.1399541973511, 'longitude': -8...",[],"{'address1': '1810 Foreman Dr', 'address2': ''...",+19313722758,(931) 372-2758,69865.781632,NaN


In [38]:
arcades_df = arcades_df.drop([])
arcades_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,UiURMNx6i-qeBWvwcIKPgQ,joyfull-arcade-spring-hill-2,Joyfull Arcade,https://s3-media2.fl.yelpcdn.com/bphoto/z5EZ5s...,False,https://www.yelp.com/biz/joyfull-arcade-spring...,1,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.77542748017264, 'longitude': -...",[],"{'address1': '2001 Campbell Station Pkwy', 'ad...",+16156141184,(615) 614-1184,55092.784846,NaN
1,8mEcWgU7q6lWchl-aJy_dw,the-city-forum-clarksville,The City Forum,https://s3-media3.fl.yelpcdn.com/bphoto/nLNur6...,False,https://www.yelp.com/biz/the-city-forum-clarks...,56,"[{'alias': 'mini_golf', 'title': 'Mini Golf'},...",4.0,"{'latitude': 36.511204, 'longitude': -87.2745814}",[],"{'address1': '2231-T Madison St', 'address2': ...",+19319195051,(931) 919-5051,33312.136770,NaN
2,_qPvx7G1-Q5I8JYuDTXlPQ,warrior-zone-fort-campbell,Warrior Zone,https://s3-media3.fl.yelpcdn.com/bphoto/KCnYC6...,False,https://www.yelp.com/biz/warrior-zone-fort-cam...,1,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",4.0,"{'latitude': 36.66417655503705, 'longitude': -...",[],"{'address1': '3910 Indiana Ave', 'address2': '...",+12704610683,(270) 461-0683,56965.972820,NaN
3,3MuUUkRYnx56VAYyENvQDA,up-down-nashville-nashville-2,Up-Down Nashville,https://s3-media3.fl.yelpcdn.com/bphoto/JuPSXB...,False,https://www.yelp.com/biz/up-down-nashville-nas...,23,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 36.1759, 'longitude': -86.7546}","[pickup, delivery]","{'address1': '927 Woodland St', 'address2': No...",+16152263747,(615) 226-3747,31732.220217,$
4,1NYbb6BzNjEyUwFKc9UfZw,game-terminal-nashville,Game Terminal,https://s3-media3.fl.yelpcdn.com/bphoto/PO3erT...,False,https://www.yelp.com/biz/game-terminal-nashvil...,62,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",4.0,"{'latitude': 36.131339492100224, 'longitude': ...",[],"{'address1': '201 Terminal Ct', 'address2': ''...",+16156102460,(615) 610-2460,34131.360677,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,ztQK4EfzwMbj3zzceiJxAA,tullahoma-bowling-lanes-tullahoma,Tullahoma Bowling Lanes,https://s3-media2.fl.yelpcdn.com/bphoto/iXGjte...,False,https://www.yelp.com/biz/tullahoma-bowling-lan...,6,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",2.5,"{'latitude': 35.3563, 'longitude': -86.19797}",[],"{'address1': '900 S Anderson St', 'address2': ...",+19314551947,(931) 455-1947,59492.132681,NaN
107,tHVupO1RIdd-DjZ-BbytWQ,ralphies-fun-center-glasgow,Ralphie's Fun Center,https://s3-media4.fl.yelpcdn.com/bphoto/yoZf_k...,False,https://www.yelp.com/biz/ralphies-fun-center-g...,9,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",4.5,"{'latitude': 37.0086209, 'longitude': -85.9189...",[],"{'address1': '702 Happy Valley Rd', 'address2'...",+12706294263,(270) 629-4263,79927.217044,NaN
108,rvimEgjXRq38ACp1wFXi_g,paradise-games-and-arcade-cookeville,Paradise Games and Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/dYpjpD...,False,https://www.yelp.com/biz/paradise-games-and-ar...,1,"[{'alias': 'videogamestores', 'title': 'Video ...",5.0,"{'latitude': 36.16233218340348, 'longitude': -...",[],"{'address1': '8 S Washington Ave', 'address2':...",+19312149263,(931) 214-9263,72450.315101,NaN
109,Jdx6joQwAE_byUINnYl1Jg,the-sk8-cookeville-2,The Sk8,https://s3-media1.fl.yelpcdn.com/bphoto/eH_2Mt...,False,https://www.yelp.com/biz/the-sk8-cookeville-2?...,4,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.5,"{'latitude': 36.1399541973511, 'longitude': -8...",[],"{'address1': '1810 Foreman Dr', 'address2': ''...",+19313722758,(931) 372-2758,69865.781632,NaN


In [40]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
family_friendly = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'family_friendly', location = location, search_limit = 20, offset = x)
        family_friendly.append(response)
        print(len(family_friendly))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [41]:
family_friendly_df = pd.DataFrame()

for family_friend in range(len(family_friendly)):
    df = pd.DataFrame(family_friendly[family_friend]['businesses'])
    family_friendly_df = family_friendly_df.append(df)
    print(family_friendly_df.shape)

(20, 16)
(40, 16)
(60, 16)
(80, 16)
(100, 16)
(120, 16)
(140, 16)
(160, 16)
(180, 16)
(200, 16)
(220, 16)
(240, 16)
(260, 16)
(280, 16)
(300, 16)
(320, 16)
(340, 16)
(360, 16)
(380, 16)
(400, 16)
(420, 16)
(440, 16)
(460, 16)
(480, 16)
(500, 16)
(520, 16)
(540, 16)
(560, 16)
(580, 16)
(600, 16)
(620, 16)
(640, 16)
(660, 16)
(680, 16)
(700, 16)
(720, 16)
(740, 16)
(760, 16)
(780, 16)
(800, 16)
(820, 16)
(840, 16)
(860, 16)
(880, 16)
(900, 16)
(920, 16)
(940, 16)
(960, 16)
(980, 16)
(1000, 16)
(1020, 16)
(1040, 16)
(1060, 16)
(1080, 16)
(1100, 16)
(1120, 16)
(1140, 16)
(1160, 16)
(1180, 16)
(1200, 16)
(1220, 16)
(1240, 16)
(1260, 16)
(1280, 16)
(1300, 16)
(1320, 16)
(1340, 16)
(1360, 16)
(1380, 16)
(1400, 16)
(1420, 16)
(1440, 16)
(1460, 16)
(1480, 16)
(1500, 16)
(1520, 16)
(1540, 16)
(1560, 16)
(1580, 16)
(1600, 16)
(1620, 16)
(1640, 16)
(1660, 16)
(1680, 16)
(1700, 16)
(1720, 16)
(1740, 16)
(1760, 16)
(1780, 16)
(1800, 16)
(1820, 16)
(1840, 16)
(1860, 16)
(1880, 16)
(1900, 16)
(1920, 1

In [42]:
family_friendly_df = family_friendly_df.drop_duplicates(subset = 'name', keep = 'first')  
family_friendly_df = family_friendly_df.reset_index(drop=True)  
family_friendly_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,UAdhMjMB-mNkY48D3LBM3w,gyro-city-ashland-city,Gyro City,https://s3-media3.fl.yelpcdn.com/bphoto/YTGpGi...,False,https://www.yelp.com/biz/gyro-city-ashland-cit...,5,"[{'alias': 'greek', 'title': 'Greek'}, {'alias...",4.5,"{'latitude': 36.27358, 'longitude': -87.06406}",[pickup],"{'address1': '164 S Main St', 'address2': 'Ste...",+16152461007,(615) 246-1007,4099.163539,NaN
1,xiK7_PdFaxg6eUpbhWoZ5g,codys-diner-ashland-city,Cody's Diner,https://s3-media1.fl.yelpcdn.com/bphoto/8WMsvr...,False,https://www.yelp.com/biz/codys-diner-ashland-c...,38,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 36.27402, 'longitude': -87.06589}",[],"{'address1': '113 Cumberland St', 'address2': ...",+16154153368,(615) 415-3368,4006.852445,$
2,AcbOk6rzLDEBqQCNQ3HCCQ,foggy-bottom-canoe-rocks-kingston-springs,Foggy Bottom Canoe Rocks,https://s3-media4.fl.yelpcdn.com/bphoto/M0VLge...,False,https://www.yelp.com/biz/foggy-bottom-canoe-ro...,46,"[{'alias': 'rafting', 'title': 'Rafting/Kayaki...",4.0,"{'latitude': 36.122811, 'longitude': -87.096378}",[],"{'address1': '1270 Hwy 70', 'address2': '', 'a...",+16159524062,(615) 952-4062,13920.101811,$$
3,DdMc-cCW9KF3Ff5H-9q93A,mama-ds-ashland-city-2,Mama D's,https://s3-media2.fl.yelpcdn.com/bphoto/TChUK2...,False,https://www.yelp.com/biz/mama-ds-ashland-city-...,16,"[{'alias': 'soup', 'title': 'Soup'}, {'alias':...",4.5,"{'latitude': 36.2748260878652, 'longitude': -8...",[],"{'address1': '116 N Main St', 'address2': '', ...",+16157929651,(615) 792-9651,4137.885771,NaN
4,IGoGsMBEEz_axpBF0exk6A,o-brien-s-southern-diner-ashland-city,O’Brien’s Southern Diner,https://s3-media2.fl.yelpcdn.com/bphoto/4R9IgL...,False,https://www.yelp.com/biz/o-brien-s-southern-di...,30,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.28057, 'longitude': -87.14424}",[],"{'address1': '2905 Hwy 49 W', 'address2': '', ...",+16158457227,(615) 845-7227,5585.342105,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573,juU0THi7QojPDnGuxlH5Sg,gyro-cafe-of-smyrna-smyrna,Gyro Cafe of Smyrna,https://s3-media2.fl.yelpcdn.com/bphoto/8NHsT2...,False,https://www.yelp.com/biz/gyro-cafe-of-smyrna-s...,20,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,"{'latitude': 35.99718, 'longitude': -86.49784}","[pickup, delivery]","{'address1': '291 Sam Ridley Pkwy E', 'address...",+16152670462,(615) 267-0462,24643.782708,$
2574,NfKcglAqZ3eZMIepH1YwYw,matt-glynn-schumacher-mortgage-nashville,Matt Glynn - Schumacher Mortgage,https://s3-media2.fl.yelpcdn.com/bphoto/a_hXVK...,False,https://www.yelp.com/biz/matt-glynn-schumacher...,62,"[{'alias': 'mortgagebrokers', 'title': 'Mortga...",5.0,"{'latitude': 36.0424826, 'longitude': -86.7699...",[],"{'address1': '1033 Demonbreun St', 'address2':...",+18588697661,(858) 869-7661,43531.957589,NaN
2575,beM1y9qBkOG_AemoZkYkhA,ramada-by-wyndham-lebanon-lebanon,Ramada by Wyndham Lebanon,https://s3-media1.fl.yelpcdn.com/bphoto/Es2931...,False,https://www.yelp.com/biz/ramada-by-wyndham-leb...,23,"[{'alias': 'hotels', 'title': 'Hotels'}]",2.0,"{'latitude': 36.188504, 'longitude': -86.297298}",[],"{'address1': '704 S Cumberland St', 'address2'...",+18002521709,(800) 252-1709,3813.242041,$
2576,S0ijTQdEURN0LLbFXnYvBQ,carlos-locksmith-nashville,Carlos Locksmith,https://s3-media1.fl.yelpcdn.com/bphoto/u-r65L...,False,https://www.yelp.com/biz/carlos-locksmith-nash...,24,"[{'alias': 'locksmiths', 'title': 'Keys & Lock...",4.5,"{'latitude': 36.0858671766346, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': '...",+16154031689,(615) 403-1689,43547.297565,NaN


In [43]:
family_friendly_df = family_friendly_df.drop([])
family_friendly_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,UAdhMjMB-mNkY48D3LBM3w,gyro-city-ashland-city,Gyro City,https://s3-media3.fl.yelpcdn.com/bphoto/YTGpGi...,False,https://www.yelp.com/biz/gyro-city-ashland-cit...,5,"[{'alias': 'greek', 'title': 'Greek'}, {'alias...",4.5,"{'latitude': 36.27358, 'longitude': -87.06406}",[pickup],"{'address1': '164 S Main St', 'address2': 'Ste...",+16152461007,(615) 246-1007,4099.163539,NaN
1,xiK7_PdFaxg6eUpbhWoZ5g,codys-diner-ashland-city,Cody's Diner,https://s3-media1.fl.yelpcdn.com/bphoto/8WMsvr...,False,https://www.yelp.com/biz/codys-diner-ashland-c...,38,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 36.27402, 'longitude': -87.06589}",[],"{'address1': '113 Cumberland St', 'address2': ...",+16154153368,(615) 415-3368,4006.852445,$
2,AcbOk6rzLDEBqQCNQ3HCCQ,foggy-bottom-canoe-rocks-kingston-springs,Foggy Bottom Canoe Rocks,https://s3-media4.fl.yelpcdn.com/bphoto/M0VLge...,False,https://www.yelp.com/biz/foggy-bottom-canoe-ro...,46,"[{'alias': 'rafting', 'title': 'Rafting/Kayaki...",4.0,"{'latitude': 36.122811, 'longitude': -87.096378}",[],"{'address1': '1270 Hwy 70', 'address2': '', 'a...",+16159524062,(615) 952-4062,13920.101811,$$
3,DdMc-cCW9KF3Ff5H-9q93A,mama-ds-ashland-city-2,Mama D's,https://s3-media2.fl.yelpcdn.com/bphoto/TChUK2...,False,https://www.yelp.com/biz/mama-ds-ashland-city-...,16,"[{'alias': 'soup', 'title': 'Soup'}, {'alias':...",4.5,"{'latitude': 36.2748260878652, 'longitude': -8...",[],"{'address1': '116 N Main St', 'address2': '', ...",+16157929651,(615) 792-9651,4137.885771,NaN
4,IGoGsMBEEz_axpBF0exk6A,o-brien-s-southern-diner-ashland-city,O’Brien’s Southern Diner,https://s3-media2.fl.yelpcdn.com/bphoto/4R9IgL...,False,https://www.yelp.com/biz/o-brien-s-southern-di...,30,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.28057, 'longitude': -87.14424}",[],"{'address1': '2905 Hwy 49 W', 'address2': '', ...",+16158457227,(615) 845-7227,5585.342105,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573,juU0THi7QojPDnGuxlH5Sg,gyro-cafe-of-smyrna-smyrna,Gyro Cafe of Smyrna,https://s3-media2.fl.yelpcdn.com/bphoto/8NHsT2...,False,https://www.yelp.com/biz/gyro-cafe-of-smyrna-s...,20,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,"{'latitude': 35.99718, 'longitude': -86.49784}","[pickup, delivery]","{'address1': '291 Sam Ridley Pkwy E', 'address...",+16152670462,(615) 267-0462,24643.782708,$
2574,NfKcglAqZ3eZMIepH1YwYw,matt-glynn-schumacher-mortgage-nashville,Matt Glynn - Schumacher Mortgage,https://s3-media2.fl.yelpcdn.com/bphoto/a_hXVK...,False,https://www.yelp.com/biz/matt-glynn-schumacher...,62,"[{'alias': 'mortgagebrokers', 'title': 'Mortga...",5.0,"{'latitude': 36.0424826, 'longitude': -86.7699...",[],"{'address1': '1033 Demonbreun St', 'address2':...",+18588697661,(858) 869-7661,43531.957589,NaN
2575,beM1y9qBkOG_AemoZkYkhA,ramada-by-wyndham-lebanon-lebanon,Ramada by Wyndham Lebanon,https://s3-media1.fl.yelpcdn.com/bphoto/Es2931...,False,https://www.yelp.com/biz/ramada-by-wyndham-leb...,23,"[{'alias': 'hotels', 'title': 'Hotels'}]",2.0,"{'latitude': 36.188504, 'longitude': -86.297298}",[],"{'address1': '704 S Cumberland St', 'address2'...",+18002521709,(800) 252-1709,3813.242041,$
2576,S0ijTQdEURN0LLbFXnYvBQ,carlos-locksmith-nashville,Carlos Locksmith,https://s3-media1.fl.yelpcdn.com/bphoto/u-r65L...,False,https://www.yelp.com/biz/carlos-locksmith-nash...,24,"[{'alias': 'locksmiths', 'title': 'Keys & Lock...",4.5,"{'latitude': 36.0858671766346, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': '...",+16154031689,(615) 403-1689,43547.297565,NaN


In [44]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
family_deals = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'family_deals', location = location, search_limit = 20, offset = x)
        family_deals.append(response)
        print(len(family_deals))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [45]:
family_deals_df = pd.DataFrame()

for family_deal in range(len(family_deals)):
    df = pd.DataFrame(family_deals[family_deal]['businesses'])
    family_deals_df = family_deals_df.append(df)
    print(family_deals_df.shape)

(20, 16)
(40, 16)
(60, 16)
(80, 16)
(100, 16)
(120, 16)
(140, 16)
(160, 16)
(180, 16)
(200, 16)
(220, 16)
(240, 16)
(260, 16)
(280, 16)
(300, 16)
(320, 16)
(340, 16)
(360, 16)
(380, 16)
(400, 16)
(420, 16)
(440, 16)
(460, 16)
(480, 16)
(500, 16)
(520, 16)
(540, 16)
(560, 16)
(580, 16)
(600, 16)
(620, 16)
(640, 16)
(660, 16)
(680, 16)
(700, 16)
(720, 16)
(740, 16)
(760, 16)
(780, 16)
(800, 16)
(820, 16)
(840, 16)
(860, 16)
(880, 16)
(900, 16)
(920, 16)
(940, 16)
(960, 16)
(980, 16)
(1000, 16)
(1020, 16)
(1040, 16)
(1060, 16)
(1080, 16)
(1100, 16)
(1120, 16)
(1140, 16)
(1160, 16)
(1180, 16)
(1200, 16)
(1220, 16)
(1240, 16)
(1260, 16)
(1280, 16)
(1300, 16)
(1320, 16)
(1340, 16)
(1360, 16)
(1380, 16)
(1400, 16)
(1420, 16)
(1440, 16)
(1460, 16)
(1480, 16)
(1500, 16)
(1520, 16)
(1540, 16)
(1560, 16)
(1580, 16)
(1600, 16)
(1620, 16)
(1640, 16)
(1660, 16)
(1680, 16)
(1700, 16)
(1720, 16)
(1740, 16)
(1760, 16)
(1780, 16)
(1800, 16)
(1820, 16)
(1840, 16)
(1860, 16)
(1880, 16)
(1900, 16)
(1920, 1

In [46]:
family_deals_df = family_deals_df.drop_duplicates(subset = 'name', keep = 'first')  
family_deals_df = family_deals_df.reset_index(drop=True)  
family_deals_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,gENQV-boJ_ykLeXMIL-6oQ,smokin-thighs-nashville-7,Smokin Thighs,https://s3-media4.fl.yelpcdn.com/bphoto/5FHIy4...,False,https://www.yelp.com/biz/smokin-thighs-nashvil...,5,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 36.15258, 'longitude': -86.83984}","[pickup, delivery]","{'address1': '4400 Charlotte Ave', 'address2':...",+16156012975,(615) 601-2975,25389.670820,NaN
1,uOx0sKON3JcIB993Ra3w7w,vuocolos-italian-restaurant-and-wine-bar-ashla...,Vuocolo's Italian Restaurant and Wine Bar,https://s3-media3.fl.yelpcdn.com/bphoto/B4aMWx...,False,https://www.yelp.com/biz/vuocolos-italian-rest...,49,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '189 Monroe Pl', 'address2': 'Ste...",+16157927447,(615) 792-7447,5573.178739,$$
2,IGoGsMBEEz_axpBF0exk6A,o-brien-s-southern-diner-ashland-city,O’Brien’s Southern Diner,https://s3-media2.fl.yelpcdn.com/bphoto/4R9IgL...,False,https://www.yelp.com/biz/o-brien-s-southern-di...,30,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.28057, 'longitude': -87.14424}",[],"{'address1': '2905 Hwy 49 W', 'address2': '', ...",+16158457227,(615) 845-7227,5585.342105,NaN
3,HHIQZQzjf6tN-QvjBJrVAw,portland-pizza-and-pub-portland-2,Portland Pizza & Pub,https://s3-media1.fl.yelpcdn.com/bphoto/50nwRB...,False,https://www.yelp.com/biz/portland-pizza-and-pu...,4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 36.60298, 'longitude': -86.5359}","[pickup, delivery]","{'address1': '819 N Broadway', 'address2': Non...",+16157455253,(615) 745-5253,63858.885037,NaN
4,JBQFWXXdKX9YKK1p3QI49w,swezeys-pub-pleasant-view-2,Swezey's Pub,https://s3-media4.fl.yelpcdn.com/bphoto/ByXE1w...,False,https://www.yelp.com/biz/swezeys-pub-pleasant-...,131,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.5,"{'latitude': 36.387752, 'longitude': -87.014627}",[],"{'address1': '254 Village Square', 'address2':...",+16153985222,(615) 398-5222,17198.197220,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,mZbLJR0_9tFE8S3s1Ixu5Q,short-mountain-distillery-woodbury-2,Short Mountain Distillery,https://s3-media2.fl.yelpcdn.com/bphoto/2Fs6nN...,False,https://www.yelp.com/biz/short-mountain-distil...,75,"[{'alias': 'distilleries', 'title': 'Distiller...",4.5,"{'latitude': 35.8669035538518, 'longitude': -8...",[],"{'address1': '8280 Short Mountain Rd', 'addres...",+16152160830,(615) 216-0830,46510.006459,$$
1081,5rXBEBszPsk31oWc0xEndA,foglight-foodhouse-walling,Foglight Foodhouse,https://s3-media2.fl.yelpcdn.com/bphoto/unfopp...,False,https://www.yelp.com/biz/foglight-foodhouse-wa...,67,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.0,"{'latitude': 35.8048110769668, 'longitude': -8...",[],"{'address1': '275 Powerhouse Rd', 'address2': ...",+19316572364,(931) 657-2364,73940.223450,$$
1082,zSkKCGi7Dq3FPU1RhRmmnw,cp-between-the-lakes-mount-juliet,CP Between The Lakes,https://s3-media3.fl.yelpcdn.com/bphoto/2SCjh8...,False,https://www.yelp.com/biz/cp-between-the-lakes-...,91,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 36.227009753061196, 'longitude': ...",[delivery],"{'address1': '4136 N Mt Juliet Rd', 'address2'...",+16157542782,(615) 754-2782,20454.214864,$$
1083,bT3pKLDsjk7JKfbVaHf9Lw,mozzarella-pizzeria-hermitage,Mozzarella Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/mNegJY...,False,https://www.yelp.com/biz/mozzarella-pizzeria-h...,117,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.194271973004, 'longitude': -86...","[delivery, pickup]","{'address1': '3912 Lebanon Pike', 'address2': ...",+16159026040,(615) 902-6040,28858.525687,$


In [47]:
family_deals_df = family_deals_df.drop([])
family_deals_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,gENQV-boJ_ykLeXMIL-6oQ,smokin-thighs-nashville-7,Smokin Thighs,https://s3-media4.fl.yelpcdn.com/bphoto/5FHIy4...,False,https://www.yelp.com/biz/smokin-thighs-nashvil...,5,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 36.15258, 'longitude': -86.83984}","[pickup, delivery]","{'address1': '4400 Charlotte Ave', 'address2':...",+16156012975,(615) 601-2975,25389.670820,NaN
1,uOx0sKON3JcIB993Ra3w7w,vuocolos-italian-restaurant-and-wine-bar-ashla...,Vuocolo's Italian Restaurant and Wine Bar,https://s3-media3.fl.yelpcdn.com/bphoto/B4aMWx...,False,https://www.yelp.com/biz/vuocolos-italian-rest...,49,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '189 Monroe Pl', 'address2': 'Ste...",+16157927447,(615) 792-7447,5573.178739,$$
2,IGoGsMBEEz_axpBF0exk6A,o-brien-s-southern-diner-ashland-city,O’Brien’s Southern Diner,https://s3-media2.fl.yelpcdn.com/bphoto/4R9IgL...,False,https://www.yelp.com/biz/o-brien-s-southern-di...,30,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.28057, 'longitude': -87.14424}",[],"{'address1': '2905 Hwy 49 W', 'address2': '', ...",+16158457227,(615) 845-7227,5585.342105,NaN
3,HHIQZQzjf6tN-QvjBJrVAw,portland-pizza-and-pub-portland-2,Portland Pizza & Pub,https://s3-media1.fl.yelpcdn.com/bphoto/50nwRB...,False,https://www.yelp.com/biz/portland-pizza-and-pu...,4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 36.60298, 'longitude': -86.5359}","[pickup, delivery]","{'address1': '819 N Broadway', 'address2': Non...",+16157455253,(615) 745-5253,63858.885037,NaN
4,JBQFWXXdKX9YKK1p3QI49w,swezeys-pub-pleasant-view-2,Swezey's Pub,https://s3-media4.fl.yelpcdn.com/bphoto/ByXE1w...,False,https://www.yelp.com/biz/swezeys-pub-pleasant-...,131,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.5,"{'latitude': 36.387752, 'longitude': -87.014627}",[],"{'address1': '254 Village Square', 'address2':...",+16153985222,(615) 398-5222,17198.197220,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,mZbLJR0_9tFE8S3s1Ixu5Q,short-mountain-distillery-woodbury-2,Short Mountain Distillery,https://s3-media2.fl.yelpcdn.com/bphoto/2Fs6nN...,False,https://www.yelp.com/biz/short-mountain-distil...,75,"[{'alias': 'distilleries', 'title': 'Distiller...",4.5,"{'latitude': 35.8669035538518, 'longitude': -8...",[],"{'address1': '8280 Short Mountain Rd', 'addres...",+16152160830,(615) 216-0830,46510.006459,$$
1081,5rXBEBszPsk31oWc0xEndA,foglight-foodhouse-walling,Foglight Foodhouse,https://s3-media2.fl.yelpcdn.com/bphoto/unfopp...,False,https://www.yelp.com/biz/foglight-foodhouse-wa...,67,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.0,"{'latitude': 35.8048110769668, 'longitude': -8...",[],"{'address1': '275 Powerhouse Rd', 'address2': ...",+19316572364,(931) 657-2364,73940.223450,$$
1082,zSkKCGi7Dq3FPU1RhRmmnw,cp-between-the-lakes-mount-juliet,CP Between The Lakes,https://s3-media3.fl.yelpcdn.com/bphoto/2SCjh8...,False,https://www.yelp.com/biz/cp-between-the-lakes-...,91,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 36.227009753061196, 'longitude': ...",[delivery],"{'address1': '4136 N Mt Juliet Rd', 'address2'...",+16157542782,(615) 754-2782,20454.214864,$$
1083,bT3pKLDsjk7JKfbVaHf9Lw,mozzarella-pizzeria-hermitage,Mozzarella Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/mNegJY...,False,https://www.yelp.com/biz/mozzarella-pizzeria-h...,117,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.194271973004, 'longitude': -86...","[delivery, pickup]","{'address1': '3912 Lebanon Pike', 'address2': ...",+16159026040,(615) 902-6040,28858.525687,$


In [48]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
children_free = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'children_free', location = location, search_limit = 20, offset = x)
        children_free.append(response)
        print(len(children_free))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


YelpAPIError: INTERNAL_ERROR: Something went wrong internally, please try again later.

In [49]:
children_free_df = pd.DataFrame()

for children_fre in range(len(children_free)):
    df = pd.DataFrame(children_free[children_fre]['businesses'])
    children_free_df = children_free_df.append(df)
    print(children_free_df.shape)

(20, 16)
(40, 16)
(60, 16)
(80, 16)
(100, 16)
(120, 16)
(140, 16)
(160, 16)
(180, 16)
(200, 16)
(220, 16)
(240, 16)
(260, 16)
(280, 16)
(300, 16)
(320, 16)
(340, 16)
(360, 16)
(380, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(408, 16)
(428, 16)
(448, 16)
(468, 16)
(488, 16)
(508, 16)
(528, 16)
(548, 16)
(568, 16)
(588, 16)
(608, 16)
(628, 16)
(648, 16)
(668, 16)
(688, 16)
(708, 16)
(728, 16)
(748, 16)
(768, 16)
(778, 16)
(778, 16)
(778, 16)
(778, 16)
(778, 16)
(778, 16)
(798, 16)
(818, 16)
(838, 16)
(858, 16)
(878, 16)
(898, 16)
(918, 16)
(938, 16)
(958, 16)
(978, 16)
(998, 16)
(1018, 16)
(1038, 16)
(1058, 16)
(1078, 16)
(1098, 16)
(1118, 16)
(1119, 16)
(1119, 16)
(1119, 16)
(1119, 16)
(1119, 16)
(1119, 16)
(1119, 16)
(1119, 16)
(1139, 16)
(1159, 16)
(1179, 16)
(1199, 16)
(1219, 16)
(1239, 16)
(1259, 16)
(1279, 16)
(1299, 16)
(1319, 16)
(1339, 16)
(1359, 16)
(1379, 16)
(1399, 16)
(1419, 16)
(1439, 16)
(1459, 16)
(1461, 16)
(1461, 16)
(1461, 16)
(1461, 16)
(1461, 16

In [50]:
children_free_df = children_free_df.drop_duplicates(subset = 'name', keep = 'first')  
children_free_df = children_free_df.reset_index(drop=True)  
children_free_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,uWIo6F9WpbW7lrNpAmlHIQ,the-children-s-playroom-drop-in-nashville,The Children’s Playroom Drop In,https://s3-media3.fl.yelpcdn.com/bphoto/OjzCDO...,False,https://www.yelp.com/biz/the-children-s-playro...,12,"[{'alias': 'childcare', 'title': 'Child Care &...",4.5,"{'latitude': 36.1037674, 'longitude': -86.8690...",[],"{'address1': '5133 Harding Pike', 'address2': ...",+16153534900,(615) 353-4900,25988.966133,NaN
1,cAIIsVrYIlBm8M4HHKqOrA,clarksville-montgomery-county-public-library-c...,Clarksville Montgomery County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/Vj0Epj...,False,https://www.yelp.com/biz/clarksville-montgomer...,16,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.521023, 'longitude': -87.340736}",[],"{'address1': '350 Pageant Ln', 'address2': 'St...",+19316488826,(931) 648-8826,37393.358988,NaN
2,IHSSqZmtM4cHMjegCOf1hw,holly-street-day-care-nashville,Holly Street Day Care,https://s3-media4.fl.yelpcdn.com/bphoto/Xf2ILb...,False,https://www.yelp.com/biz/holly-street-day-care...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.1761436462402, 'longitude': -8...",[],"{'address1': '1401 Holly St', 'address2': '', ...",+16152278252,(615) 227-8252,32494.885223,NaN
3,Jhh-nbZF0wNnvaunKdxp6Q,the-childrens-playroom-drop-in-brentwood-10,The Children's Playroom Drop In,https://s3-media2.fl.yelpcdn.com/bphoto/wOH3aR...,False,https://www.yelp.com/biz/the-childrens-playroo...,10,"[{'alias': 'childcare', 'title': 'Child Care &...",4.0,"{'latitude': 36.02499626222008, 'longitude': -...",[],"{'address1': '330 Franklin Rd', 'address2': 'S...",+16153715279,(615) 371-5279,37061.753124,NaN
4,pIRsnvsQxTvX-cf9THTRBA,little-professional-learning-center-nashville,Little Professional Learning Center,https://s3-media1.fl.yelpcdn.com/bphoto/w1YOie...,False,https://www.yelp.com/biz/little-professional-l...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.1703606, 'longitude': -86.6740...",[],"{'address1': '228 Shady Grove Dr', 'address2':...",+16159020777,(615) 902-0777,38925.204170,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,q-7cQQPae0KmuRv2saElQQ,stardust-drive-in-theatre-watertown,Stardust Drive-In Theatre,https://s3-media3.fl.yelpcdn.com/bphoto/HhIPiQ...,False,https://www.yelp.com/biz/stardust-drive-in-the...,63,"[{'alias': 'driveintheater', 'title': 'Drive-I...",4.5,"{'latitude': 36.1016616821289, 'longitude': -8...",[],"{'address1': '310 Purple Tiger Dr', 'address2'...",+16152370077,(615) 237-0077,56806.945844,NaN
382,qEfkQtNXZkmZaRUtZ6-yvg,land-between-the-lakes-golden-pond,Land Between the Lakes,https://s3-media3.fl.yelpcdn.com/bphoto/dAB1Sc...,False,https://www.yelp.com/biz/land-between-the-lake...,18,"[{'alias': 'recreation', 'title': 'Recreation ...",4.0,"{'latitude': 36.778591154122, 'longitude': -88...",[],"{'address1': '238 Visitor Center Dr', 'address...",+18005257077,(800) 525-7077,94398.433737,NaN
383,UX3WjoVUIgy7qXaVGun_Wg,jack-daniel-lynchburg,The Jack Daniel Distillery,https://s3-media1.fl.yelpcdn.com/bphoto/UC19KM...,False,https://www.yelp.com/biz/jack-daniel-lynchburg...,962,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.5,"{'latitude': 35.284578, 'longitude': -86.371815}",[],"{'address1': '133 Lynchburg Hwy', 'address2': ...",+19317596357,(931) 759-6357,73717.409897,$$
384,rYJ5afN6yVvgrKxRtPfKEA,fayetteville-lincoln-county-public-library-fay...,Fayetteville-Lincoln County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/04TWyp...,False,https://www.yelp.com/biz/fayetteville-lincoln-...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.15466, 'longitude': -86.56921}",[],"{'address1': '306 Elk Ave N', 'address2': '', ...",+19314333286,(931) 433-3286,69453.871215,NaN


In [51]:
children_free_df = children_free_df.drop([])
children_free_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,uWIo6F9WpbW7lrNpAmlHIQ,the-children-s-playroom-drop-in-nashville,The Children’s Playroom Drop In,https://s3-media3.fl.yelpcdn.com/bphoto/OjzCDO...,False,https://www.yelp.com/biz/the-children-s-playro...,12,"[{'alias': 'childcare', 'title': 'Child Care &...",4.5,"{'latitude': 36.1037674, 'longitude': -86.8690...",[],"{'address1': '5133 Harding Pike', 'address2': ...",+16153534900,(615) 353-4900,25988.966133,NaN
1,cAIIsVrYIlBm8M4HHKqOrA,clarksville-montgomery-county-public-library-c...,Clarksville Montgomery County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/Vj0Epj...,False,https://www.yelp.com/biz/clarksville-montgomer...,16,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.521023, 'longitude': -87.340736}",[],"{'address1': '350 Pageant Ln', 'address2': 'St...",+19316488826,(931) 648-8826,37393.358988,NaN
2,IHSSqZmtM4cHMjegCOf1hw,holly-street-day-care-nashville,Holly Street Day Care,https://s3-media4.fl.yelpcdn.com/bphoto/Xf2ILb...,False,https://www.yelp.com/biz/holly-street-day-care...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.1761436462402, 'longitude': -8...",[],"{'address1': '1401 Holly St', 'address2': '', ...",+16152278252,(615) 227-8252,32494.885223,NaN
3,Jhh-nbZF0wNnvaunKdxp6Q,the-childrens-playroom-drop-in-brentwood-10,The Children's Playroom Drop In,https://s3-media2.fl.yelpcdn.com/bphoto/wOH3aR...,False,https://www.yelp.com/biz/the-childrens-playroo...,10,"[{'alias': 'childcare', 'title': 'Child Care &...",4.0,"{'latitude': 36.02499626222008, 'longitude': -...",[],"{'address1': '330 Franklin Rd', 'address2': 'S...",+16153715279,(615) 371-5279,37061.753124,NaN
4,pIRsnvsQxTvX-cf9THTRBA,little-professional-learning-center-nashville,Little Professional Learning Center,https://s3-media1.fl.yelpcdn.com/bphoto/w1YOie...,False,https://www.yelp.com/biz/little-professional-l...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.1703606, 'longitude': -86.6740...",[],"{'address1': '228 Shady Grove Dr', 'address2':...",+16159020777,(615) 902-0777,38925.204170,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,q-7cQQPae0KmuRv2saElQQ,stardust-drive-in-theatre-watertown,Stardust Drive-In Theatre,https://s3-media3.fl.yelpcdn.com/bphoto/HhIPiQ...,False,https://www.yelp.com/biz/stardust-drive-in-the...,63,"[{'alias': 'driveintheater', 'title': 'Drive-I...",4.5,"{'latitude': 36.1016616821289, 'longitude': -8...",[],"{'address1': '310 Purple Tiger Dr', 'address2'...",+16152370077,(615) 237-0077,56806.945844,NaN
382,qEfkQtNXZkmZaRUtZ6-yvg,land-between-the-lakes-golden-pond,Land Between the Lakes,https://s3-media3.fl.yelpcdn.com/bphoto/dAB1Sc...,False,https://www.yelp.com/biz/land-between-the-lake...,18,"[{'alias': 'recreation', 'title': 'Recreation ...",4.0,"{'latitude': 36.778591154122, 'longitude': -88...",[],"{'address1': '238 Visitor Center Dr', 'address...",+18005257077,(800) 525-7077,94398.433737,NaN
383,UX3WjoVUIgy7qXaVGun_Wg,jack-daniel-lynchburg,The Jack Daniel Distillery,https://s3-media1.fl.yelpcdn.com/bphoto/UC19KM...,False,https://www.yelp.com/biz/jack-daniel-lynchburg...,962,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.5,"{'latitude': 35.284578, 'longitude': -86.371815}",[],"{'address1': '133 Lynchburg Hwy', 'address2': ...",+19317596357,(931) 759-6357,73717.409897,$$
384,rYJ5afN6yVvgrKxRtPfKEA,fayetteville-lincoln-county-public-library-fay...,Fayetteville-Lincoln County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/04TWyp...,False,https://www.yelp.com/biz/fayetteville-lincoln-...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.15466, 'longitude': -86.56921}",[],"{'address1': '306 Elk Ave N', 'address2': '', ...",+19314333286,(931) 433-3286,69453.871215,NaN


# search landmarks and libraries and family friendly and family deals or children free and arcades